In [ ]:
%%configure -f
{
"kind": "pyspark",

"conf": {
   
     "spark.master": "yarn",
    "spark.yarn.dist.archives": "/use_case/hhmatching/Library_zip/extra_lib_3.zip#lcm_zip", 
    "spark.yarn.appMasterEnv.PYSPARK_PYTHON": "./lcm_zip/bin/python",
    "spark.driver.cores": "8", 
    "spark.driver.memory": "13000m",  
    "spark.executor.memory": "13000m",
    "spark.executor.cores": "8", 
     "spark.dynamicAllocation.enabled": "true",
     "spark.dynamicAllocation.minExecutors": "3",
    
    "NotebookApp.iopub_data_rate_limit":"1000000000.0",
    "NotebookApp.rate_limit_window":"3.0"
    
      
    }
}

### Import Libraries<a name="libs"></a>
<a href = "#toc">Back to top</a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import dataframe
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import lit,log
from pyspark.sql.functions import isnan
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import array, col, explode, lit, struct
from pyspark.sql.functions import col, lit, coalesce, greatest, concat
from pyspark.sql.functions import regexp_replace 
from pyspark.sql import DataFrame
from pyspark.sql.window import *
from pyspark.sql.functions import row_number
from typing import Iterable
import pandas as pd
import numpy as np
from pyspark.sql.types import StringType,FloatType,DoubleType,LongType,IntegerType,ArrayType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import Bucketizer
import pandas as pd, numpy as np
from matplotlib import pyplot as plt
#from plotly.offline import iplot
#import plotly.graph_objs as go
#%matplotlib inline
import seaborn as sns
from pandas.plotting import table
#from sklearn.preprocessing import LabelEncoder
import matplotlib.patches as mpatches

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder 
#from sklearn.metrics import roc_curve, auc
#from sklearn.metrics import log_loss
from pyspark.sql import Window
from pyspark.sql.functions import rank,sum,col
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorSlicer

from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.sql.functions import when
from pyspark.sql import functions as F
from pyspark.sql.functions import avg
from pyspark.ml import Pipeline
# Initiate Spark Session
# spark = SparkSession\
#     .builder\
#     .appName("example-spark")\
#     .config("spark.sql.crossJoin.enabled","true")\
#     .getOrCreate()

# ## Creating Hive Context

# sc = pyspark.SparkContext()
# hiveContext = HiveContext(sc)

#spark = SparkSession.builder\
# .master("local[2]")\
# .config("spark.submit.deployMode", "client")\
# .config("spark.eventLog.enabled", "true")\
# .config("spark.dynamicAllocation.enabled", "true")\
# .config("spark.dynamicAllocation.executorIdleTimeout", "1500s")\
# .config("spark.dynamicAllocation.initialExecutors", "53")\
# .appName("Base_Line")\
# .getOrCreate()
#
#sc = spark.sparkContext
#L = spark.sparkContext._conf.getAll()

#bi_reason = sys.argv[0]

#conf = SparkConf().setAppName('Base_Line')
#sc = SparkContext(conf=conf)

#sqlContext = HiveContext(sc)

spark.conf.set("spark.sql.execution.arrow.enabled","false")

spark.conf.set("spark.sql.execution.arrow.fallback.enabled","true")

from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

from pyspark.sql import HiveContext
hiveContext = HiveContext(spark)


In [ ]:
def dataImport(db,table):
    df = spark.sql("select * from {0}.{1}".format(db,table))
    return df

In [ ]:
df_2p = dataImport("sbx_adhoc_ds","2p_scoring_eop202101_20210222_w_consent")
df_3p = dataImport("sbx_adhoc_ds","3p_scoring_eop202101_20210222_w_consent")


In [ ]:
df = df_2p.union(df_3p.select(df_2p.columns))

In [ ]:
df = df.withColumnRenamed("oib_kupac","oib")

In [ ]:
df1 = df.filter(F.col("alls_period") == "202101")

In [ ]:
df1.count()

In [ ]:
L = ["MAXTV_TENURE",
"TOTAL_FAULT_DAYS_DIF_P456_P123",
"TOTAL_FAULT_DAYS_DIF_P789_P123",
"TOTAL_FAULT_DAYS_DIF_P789_P456",
"TOTAL_FAULTS_DIFF_P456_P123",
"TOTAL_FAULTS_DIFF_P789_P123",
"TOTAL_FAULTS_DIFF_P789_P456",
"CC_ALL_COUNT_LAST_3_MONTHS",
"CC_ALL_COUNT_LAST_6_MONHTS",
"CC_ALL_COUNT_LAST_9_MONTHS",
"CC_ALL_DIFF_P456_P123",
"CC_ALL_DIFF_P789_P123",
"CC_ALL_DIFF_P789_P456",
"HT_RES_SHARE_ADM_DIFF_P3_P1",
"HT_RES_SHARE_ADM_DIFF_P5_P1",
"HT_RES_SHARE_ADM_DIFF_P7_P1",
"HT_RES_SHARE_ADM_DIFF_P9_P1",
"HT_RES_SHARE_IN_ADM",
"AVG_BB_TRAFFIC_DIFF_P456_P123",
"AVG_BB_TRAFFIC_DIFF_P789_P123",
"AVG_BB_TRAFFIC_DIFF_P789_P456",
"AVG_BB_TRAFFIC_P123",
"AVG_BB_TRAFFIC_P456",
"AVG_BB_TRAFFIC_P789",
"AVG_MINUTES_DIFF_P456_P123",
"AVG_MINUTES_DIFF_P789_P123",
"AVG_MINUTES_DIFF_P789_P456",
"AVG_NR_OF_CALLS_DIFF_P456_P123",
"AVG_NR_OF_CALLS_DIFF_P789_P123",
"AVG_NR_OF_CALLS_DIFF_P789_P456"]

In [ ]:
df2 = df1.drop_duplicates(["customer_play_id","oib"])

In [ ]:
df3 = df2.select("oib",*L)

In [ ]:
df3 = df3.withColumn("MAXTV_TENURE",F.col("MAXTV_TENURE").cast(IntegerType()))
df3 = df3.withColumn("TOTAL_FAULT_DAYS_DIF_P456_P123",F.col("TOTAL_FAULT_DAYS_DIF_P456_P123").cast(IntegerType()))
df3 = df3.withColumn("TOTAL_FAULT_DAYS_DIF_P789_P123",F.col("TOTAL_FAULT_DAYS_DIF_P789_P123").cast(IntegerType()))
df3 = df3.withColumn("TOTAL_FAULT_DAYS_DIF_P789_P456",F.col("TOTAL_FAULT_DAYS_DIF_P789_P456").cast(IntegerType()))
df3 = df3.withColumn("TOTAL_FAULTS_DIFF_P456_P123",F.col("TOTAL_FAULTS_DIFF_P456_P123").cast(IntegerType()))
df3 = df3.withColumn("TOTAL_FAULTS_DIFF_P789_P123",F.col("TOTAL_FAULTS_DIFF_P789_P123").cast(IntegerType()))
df3 = df3.withColumn("TOTAL_FAULTS_DIFF_P789_P456",F.col("TOTAL_FAULTS_DIFF_P789_P456").cast(IntegerType()))
df3 = df3.withColumn("CC_ALL_COUNT_LAST_3_MONTHS",F.col("CC_ALL_COUNT_LAST_3_MONTHS").cast(IntegerType()))
df3 = df3.withColumn("CC_ALL_COUNT_LAST_6_MONHTS",F.col("CC_ALL_COUNT_LAST_6_MONHTS").cast(IntegerType()))
df3 = df3.withColumn("CC_ALL_COUNT_LAST_9_MONTHS",F.col("CC_ALL_COUNT_LAST_9_MONTHS").cast(IntegerType()))
df3 = df3.withColumn("CC_ALL_DIFF_P456_P123",F.col("CC_ALL_DIFF_P456_P123").cast(IntegerType()))
df3 = df3.withColumn("CC_ALL_DIFF_P789_P123",F.col("CC_ALL_DIFF_P789_P123").cast(IntegerType()))
df3 = df3.withColumn("CC_ALL_DIFF_P789_P456",F.col("CC_ALL_DIFF_P789_P456").cast(IntegerType()))
df3 = df3.withColumn("HT_RES_SHARE_ADM_DIFF_P3_P1",F.col("HT_RES_SHARE_ADM_DIFF_P3_P1").cast(IntegerType()))
df3 = df3.withColumn("HT_RES_SHARE_ADM_DIFF_P5_P1",F.col("HT_RES_SHARE_ADM_DIFF_P5_P1").cast(IntegerType()))
df3 = df3.withColumn("HT_RES_SHARE_ADM_DIFF_P7_P1",F.col("HT_RES_SHARE_ADM_DIFF_P7_P1").cast(IntegerType()))
df3 = df3.withColumn("HT_RES_SHARE_ADM_DIFF_P9_P1",F.col("HT_RES_SHARE_ADM_DIFF_P9_P1").cast(IntegerType()))
df3 = df3.withColumn("HT_RES_SHARE_IN_ADM",F.col("HT_RES_SHARE_IN_ADM").cast(IntegerType()))
df3 = df3.withColumn("AVG_BB_TRAFFIC_DIFF_P456_P123",F.col("AVG_BB_TRAFFIC_DIFF_P456_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_BB_TRAFFIC_DIFF_P789_P123",F.col("AVG_BB_TRAFFIC_DIFF_P789_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_BB_TRAFFIC_DIFF_P789_P456",F.col("AVG_BB_TRAFFIC_DIFF_P789_P456").cast(IntegerType()))
df3 = df3.withColumn("AVG_BB_TRAFFIC_P123",F.col("AVG_BB_TRAFFIC_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_BB_TRAFFIC_P456",F.col("AVG_BB_TRAFFIC_P456").cast(IntegerType()))
df3 = df3.withColumn("AVG_BB_TRAFFIC_P789",F.col("AVG_BB_TRAFFIC_P789").cast(IntegerType()))
df3 = df3.withColumn("AVG_MINUTES_DIFF_P456_P123",F.col("AVG_MINUTES_DIFF_P456_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_MINUTES_DIFF_P789_P123",F.col("AVG_MINUTES_DIFF_P789_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_MINUTES_DIFF_P789_P456",F.col("AVG_MINUTES_DIFF_P789_P456").cast(IntegerType()))
df3 = df3.withColumn("AVG_NR_OF_CALLS_DIFF_P456_P123",F.col("AVG_NR_OF_CALLS_DIFF_P456_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_NR_OF_CALLS_DIFF_P789_P123",F.col("AVG_NR_OF_CALLS_DIFF_P789_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_NR_OF_CALLS_DIFF_P789_P456",F.col("AVG_NR_OF_CALLS_DIFF_P789_P456").cast(IntegerType()))

In [ ]:
base1 = df3.groupBy("oib").agg(F.max("MAXTV_TENURE").alias("MAXTV_TENURE"),
F.max("TOTAL_FAULT_DAYS_DIF_P456_P123").alias("TOTAL_FAULT_DAYS_DIF_P456_P123"),
F.max("TOTAL_FAULT_DAYS_DIF_P789_P123").alias("TOTAL_FAULT_DAYS_DIF_P789_P123"),
F.max("TOTAL_FAULT_DAYS_DIF_P789_P456").alias("TOTAL_FAULT_DAYS_DIF_P789_P456"),
F.max("TOTAL_FAULTS_DIFF_P456_P123").alias("TOTAL_FAULTS_DIFF_P456_P123"),
F.max("TOTAL_FAULTS_DIFF_P789_P123").alias("TOTAL_FAULTS_DIFF_P789_P123"),
F.max("TOTAL_FAULTS_DIFF_P789_P456").alias("TOTAL_FAULTS_DIFF_P789_P456"),
F.max("CC_ALL_COUNT_LAST_3_MONTHS").alias("CC_ALL_COUNT_LAST_3_MONTHS"),
F.max("CC_ALL_COUNT_LAST_6_MONHTS").alias("CC_ALL_COUNT_LAST_6_MONHTS"),
F.max("CC_ALL_COUNT_LAST_9_MONTHS").alias("CC_ALL_COUNT_LAST_9_MONTHS"),
F.max("CC_ALL_DIFF_P456_P123").alias("CC_ALL_DIFF_P456_P123"),
F.max("CC_ALL_DIFF_P789_P123").alias("CC_ALL_DIFF_P789_P123"),
F.max("CC_ALL_DIFF_P789_P456").alias("CC_ALL_DIFF_P789_P456"),
F.max("HT_RES_SHARE_ADM_DIFF_P3_P1").alias("HT_RES_SHARE_ADM_DIFF_P3_P1"),
F.max("HT_RES_SHARE_ADM_DIFF_P5_P1").alias("HT_RES_SHARE_ADM_DIFF_P5_P1"),
F.max("HT_RES_SHARE_ADM_DIFF_P7_P1").alias("HT_RES_SHARE_ADM_DIFF_P7_P1"),
F.max("HT_RES_SHARE_ADM_DIFF_P9_P1").alias("HT_RES_SHARE_ADM_DIFF_P9_P1"),
F.max("HT_RES_SHARE_IN_ADM").alias("HT_RES_SHARE_IN_ADM"),
F.max("AVG_BB_TRAFFIC_DIFF_P456_P123").alias("AVG_BB_TRAFFIC_DIFF_P456_P123"),
F.max("AVG_BB_TRAFFIC_DIFF_P789_P123").alias("AVG_BB_TRAFFIC_DIFF_P789_P123"),
F.max("AVG_BB_TRAFFIC_DIFF_P789_P456").alias("AVG_BB_TRAFFIC_DIFF_P789_P456"),
F.max("AVG_BB_TRAFFIC_P123").alias("AVG_BB_TRAFFIC_P123"),
F.max("AVG_BB_TRAFFIC_P456").alias("AVG_BB_TRAFFIC_P456"),
F.max("AVG_BB_TRAFFIC_P789").alias("AVG_BB_TRAFFIC_P789"),
F.max("AVG_MINUTES_DIFF_P456_P123").alias("AVG_MINUTES_DIFF_P456_P123"),
F.max("AVG_MINUTES_DIFF_P789_P123").alias("AVG_MINUTES_DIFF_P789_P123"),
F.max("AVG_MINUTES_DIFF_P789_P456").alias("AVG_MINUTES_DIFF_P789_P456"),
F.max("AVG_NR_OF_CALLS_DIFF_P456_P123").alias("AVG_NR_OF_CALLS_DIFF_P456_P123"),
F.max("AVG_NR_OF_CALLS_DIFF_P789_P123").alias("AVG_NR_OF_CALLS_DIFF_P789_P123"),
F.max("AVG_NR_OF_CALLS_DIFF_P789_P456").alias("AVG_NR_OF_CALLS_DIFF_P789_P456"))

In [ ]:
base1.select("oib","AVG_NR_OF_CALLS_DIFF_P456_P123").count()

In [ ]:
L1 = ["HALO_DAY_TILL_CNTRCT_EXP",
"MAXTV_DAY_TILL_CNTRCT_EXP",
"BB_SPEED_CLUSTER",
"EKVIVALENTNA_UDALJENOST",
"MAX_ADSL_ONLY_CLUSTER",
"MAX_ADSL_W_IPTV_CLUSTER",
]

In [ ]:
df4 = df2.select("oib",*L1)

In [ ]:
df4 = df4.withColumn("HALO_DAY_TILL_CNTRCT_EXP",F.col("HALO_DAY_TILL_CNTRCT_EXP").cast(IntegerType()))
df4 = df4.withColumn("MAXTV_DAY_TILL_CNTRCT_EXP",F.col("MAXTV_DAY_TILL_CNTRCT_EXP").cast(IntegerType()))
df4 = df4.withColumn("BB_SPEED_CLUSTER",F.col("BB_SPEED_CLUSTER").cast(IntegerType()))
df4 = df4.withColumn("EKVIVALENTNA_UDALJENOST",F.col("EKVIVALENTNA_UDALJENOST").cast(IntegerType()))
df4 = df4.withColumn("MAX_ADSL_ONLY_CLUSTER",F.col("MAX_ADSL_ONLY_CLUSTER").cast(IntegerType()))
df4 = df4.withColumn("MAX_ADSL_W_IPTV_CLUSTER",F.col("MAX_ADSL_W_IPTV_CLUSTER").cast(IntegerType()))


In [ ]:
base2 = df4.groupBy("oib").agg(
F.min("HALO_DAY_TILL_CNTRCT_EXP").alias("HALO_DAY_TILL_CNTRCT_EXP"),
F.min("MAXTV_DAY_TILL_CNTRCT_EXP").alias("MAXTV_DAY_TILL_CNTRCT_EXP"),
F.min("BB_SPEED_CLUSTER").alias("BB_SPEED_CLUSTER"),
F.min("EKVIVALENTNA_UDALJENOST").alias("EKVIVALENTNA_UDALJENOST"),
F.min("MAX_ADSL_ONLY_CLUSTER").alias("MAX_ADSL_ONLY_CLUSTER"),
F.min("MAX_ADSL_W_IPTV_CLUSTER").alias("MAX_ADSL_W_IPTV_CLUSTER"))

In [ ]:
base2.count()

In [ ]:
L2 = ["COMPLAINT_COUNT_LAST_3_MONTHS",
"COMPLAINT_COUNT_LAST_6_MONHTS",
"COMPLAINT_COUNT_LAST_9_MONTHS",
"COMPLAINT_DIFF_P456_P123",
"COMPLAINT_DIFF_P789_P123",
"COMPLAINT_DIFF_P789_P456",
"DAYS_IN_FAULT_LAST_NINE_MNTHS",
"DAYS_IN_FAULT_LAST_SIX_MNTHS",
"DAYS_IN_FAULT_LAST_THREE_MNTHS",
"FAULTS_LAST_FIX_MONHTS",
"FAULTS_LAST_NINE_MONTHS",
"FAULTS_LAST_THREE_MONTHS"]


In [ ]:
df5 = df2.select("oib",*L2)

In [ ]:
df5 = df5.withColumn("COMPLAINT_COUNT_LAST_3_MONTHS",F.col("COMPLAINT_COUNT_LAST_3_MONTHS").cast(IntegerType()))
df5 = df5.withColumn("COMPLAINT_COUNT_LAST_6_MONHTS",F.col("COMPLAINT_COUNT_LAST_6_MONHTS").cast(IntegerType()))
df5 = df5.withColumn("COMPLAINT_COUNT_LAST_9_MONTHS",F.col("COMPLAINT_COUNT_LAST_9_MONTHS").cast(IntegerType()))
df5 = df5.withColumn("COMPLAINT_DIFF_P456_P123",F.col("COMPLAINT_DIFF_P456_P123").cast(IntegerType()))
df5 = df5.withColumn("COMPLAINT_DIFF_P789_P123",F.col("COMPLAINT_DIFF_P789_P123").cast(IntegerType()))
df5 = df5.withColumn("COMPLAINT_DIFF_P789_P456",F.col("COMPLAINT_DIFF_P789_P456").cast(IntegerType()))
df5 = df5.withColumn("DAYS_IN_FAULT_LAST_NINE_MNTHS",F.col("DAYS_IN_FAULT_LAST_NINE_MNTHS").cast(IntegerType()))
df5 = df5.withColumn("DAYS_IN_FAULT_LAST_SIX_MNTHS",F.col("DAYS_IN_FAULT_LAST_SIX_MNTHS").cast(IntegerType()))
df5 = df5.withColumn("DAYS_IN_FAULT_LAST_THREE_MNTHS",F.col("DAYS_IN_FAULT_LAST_THREE_MNTHS").cast(IntegerType()))
df5 = df5.withColumn("FAULTS_LAST_FIX_MONHTS",F.col("FAULTS_LAST_FIX_MONHTS").cast(IntegerType()))
df5 = df5.withColumn("FAULTS_LAST_NINE_MONTHS",F.col("FAULTS_LAST_NINE_MONTHS").cast(IntegerType()))
df5 = df5.withColumn("FAULTS_LAST_THREE_MONTHS",F.col("FAULTS_LAST_THREE_MONTHS").cast(IntegerType()))

In [ ]:

base3 = df5.groupBy("oib").agg(
F.sum("COMPLAINT_COUNT_LAST_3_MONTHS").alias("COMPLAINT_COUNT_LAST_3_MONTHS"),
F.sum("COMPLAINT_COUNT_LAST_6_MONHTS").alias("COMPLAINT_COUNT_LAST_6_MONHTS"),
F.sum("COMPLAINT_COUNT_LAST_9_MONTHS").alias("COMPLAINT_COUNT_LAST_9_MONTHS"),
F.sum("COMPLAINT_DIFF_P456_P123").alias("COMPLAINT_DIFF_P456_P123"),
F.sum("COMPLAINT_DIFF_P789_P123").alias("COMPLAINT_DIFF_P789_P123"),
F.sum("COMPLAINT_DIFF_P789_P456").alias("COMPLAINT_DIFF_P789_P456"),
F.sum("DAYS_IN_FAULT_LAST_NINE_MNTHS").alias("DAYS_IN_FAULT_LAST_NINE_MNTHS"),
F.sum("DAYS_IN_FAULT_LAST_SIX_MNTHS").alias("DAYS_IN_FAULT_LAST_SIX_MNTHS"),
F.sum("DAYS_IN_FAULT_LAST_THREE_MNTHS").alias("DAYS_IN_FAULT_LAST_THREE_MNTHS"),
F.sum("FAULTS_LAST_FIX_MONHTS").alias("FAULTS_LAST_FIX_MONHTS"),
F.sum("FAULTS_LAST_NINE_MONTHS").alias("FAULTS_LAST_NINE_MONTHS"),
F.sum("FAULTS_LAST_THREE_MONTHS").alias("FAULTS_LAST_THREE_MONTHS"))

In [ ]:
L3 = ["HH_SPEND_ON_NUM",
"RATE_HH_SUM_ARPA_M1_AVG_M23",
"RATE_HH_SUM_ARPA_M2_AVG_M31",
"RATE_HH_SUM_ARPA_M3_AVG_M12",
"RATE_HH_SUM_ARPA_M7_AVG_M89",
"RATE_HH_SUM_ARPA_M8_AVG_M97",
"RATE_HH_SUM_ARPA_M9_AVG_M78",
"RATE_SUM_ARPA_M4_AVG_M56",
"RATE_SUM_ARPA_M5_AVG_M64",
"RATE_SUM_ARPA_M6_AVG_M45",
"SUM_ARPA_HH_MIN_1",
"SUM_ARPA_HH_MIN_2",
"SUM_ARPA_HH_MIN_3",
"SUM_ARPA_HH_MIN_4",
"SUM_ARPA_HH_MIN_5",
"SUM_ARPA_HH_MIN_6",
"SUM_ARPA_HH_MIN_7",
"SUM_ARPA_HH_MIN_8",
"SUM_ARPA_HH_MIN_9",
"SUM_ARPA_TOT_HH_MIN_123",
"SUM_ARPA_TOT_HH_MIN_456",
"SUM_ARPA_TOT_HH_MIN_789",
"AVG_TOT_PAY_DIFF_P456_P123",
"AVG_TOT_PAY_DIFF_P789_P123",
"AVG_TOT_PAY_DIFF_P789_P456",
"AVG_TOTAL_PAYAMOUNT_P123",
"AVG_TOTAL_PAYAMOUNT_P456",
"AVG_TOTAL_PAYAMOUNT_P789",
"TOTAL_BILLED_FOR_LAST_3_MONTHS",
"TOTAL_BILLED_FOR_LAST_6_MONTHS",
"TOTAL_BILLED_FOR_LAST_9_MONTHS",
"AVG_MINUTES_P123",
"AVG_MINUTES_P456",
"AVG_MINUTES_P789",
"AVG_NR_OF_CALLS_P123",
"AVG_NR_OF_CALLS_P456",
"AVG_NR_OF_CALLS_P789"
]

In [ ]:
df6 = df2.select("oib",*L3)

In [ ]:
df6 = df6.withColumn("HH_SPEND_ON_NUM",F.col("HH_SPEND_ON_NUM").cast(IntegerType()))
df6 = df6.withColumn("RATE_HH_SUM_ARPA_M1_AVG_M23",F.col("RATE_HH_SUM_ARPA_M1_AVG_M23").cast(IntegerType()))
df6 = df6.withColumn("RATE_HH_SUM_ARPA_M2_AVG_M31",F.col("RATE_HH_SUM_ARPA_M2_AVG_M31").cast(IntegerType()))
df6 = df6.withColumn("RATE_HH_SUM_ARPA_M3_AVG_M12",F.col("RATE_HH_SUM_ARPA_M3_AVG_M12").cast(IntegerType()))
df6 = df6.withColumn("RATE_HH_SUM_ARPA_M7_AVG_M89",F.col("RATE_HH_SUM_ARPA_M7_AVG_M89").cast(IntegerType()))
df6 = df6.withColumn("RATE_HH_SUM_ARPA_M8_AVG_M97",F.col("RATE_HH_SUM_ARPA_M8_AVG_M97").cast(IntegerType()))
df6 = df6.withColumn("RATE_HH_SUM_ARPA_M9_AVG_M78",F.col("RATE_HH_SUM_ARPA_M9_AVG_M78").cast(IntegerType()))
df6 = df6.withColumn("RATE_SUM_ARPA_M4_AVG_M56",F.col("RATE_SUM_ARPA_M4_AVG_M56").cast(IntegerType()))
df6 = df6.withColumn("RATE_SUM_ARPA_M5_AVG_M64",F.col("RATE_SUM_ARPA_M5_AVG_M64").cast(IntegerType()))
df6 = df6.withColumn("RATE_SUM_ARPA_M6_AVG_M45",F.col("RATE_SUM_ARPA_M6_AVG_M45").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_1",F.col("SUM_ARPA_HH_MIN_1").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_2",F.col("SUM_ARPA_HH_MIN_2").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_3",F.col("SUM_ARPA_HH_MIN_3").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_4",F.col("SUM_ARPA_HH_MIN_4").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_5",F.col("SUM_ARPA_HH_MIN_5").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_6",F.col("SUM_ARPA_HH_MIN_6").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_7",F.col("SUM_ARPA_HH_MIN_7").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_8",F.col("SUM_ARPA_HH_MIN_8").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_9",F.col("SUM_ARPA_HH_MIN_9").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_TOT_HH_MIN_123",F.col("SUM_ARPA_TOT_HH_MIN_123").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_TOT_HH_MIN_456",F.col("SUM_ARPA_TOT_HH_MIN_456").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_TOT_HH_MIN_789",F.col("SUM_ARPA_TOT_HH_MIN_789").cast(IntegerType()))
df6 = df6.withColumn("AVG_TOT_PAY_DIFF_P456_P123",F.col("AVG_TOT_PAY_DIFF_P456_P123").cast(IntegerType()))
df6 = df6.withColumn("AVG_TOT_PAY_DIFF_P789_P123",F.col("AVG_TOT_PAY_DIFF_P789_P123").cast(IntegerType()))
df6 = df6.withColumn("AVG_TOT_PAY_DIFF_P789_P456",F.col("AVG_TOT_PAY_DIFF_P789_P456").cast(IntegerType()))
df6 = df6.withColumn("AVG_TOTAL_PAYAMOUNT_P123",F.col("AVG_TOTAL_PAYAMOUNT_P123").cast(IntegerType()))
df6 = df6.withColumn("AVG_TOTAL_PAYAMOUNT_P456",F.col("AVG_TOTAL_PAYAMOUNT_P456").cast(IntegerType()))
df6 = df6.withColumn("AVG_TOTAL_PAYAMOUNT_P789",F.col("AVG_TOTAL_PAYAMOUNT_P789").cast(IntegerType()))
df6 = df6.withColumn("TOTAL_BILLED_FOR_LAST_3_MONTHS",F.col("TOTAL_BILLED_FOR_LAST_3_MONTHS").cast(IntegerType()))
df6 = df6.withColumn("TOTAL_BILLED_FOR_LAST_6_MONTHS",F.col("TOTAL_BILLED_FOR_LAST_6_MONTHS").cast(IntegerType()))
df6 = df6.withColumn("TOTAL_BILLED_FOR_LAST_9_MONTHS",F.col("TOTAL_BILLED_FOR_LAST_9_MONTHS").cast(IntegerType()))
df6 = df6.withColumn("AVG_MINUTES_P123",F.col("AVG_MINUTES_P123").cast(IntegerType()))
df6 = df6.withColumn("AVG_MINUTES_P456",F.col("AVG_MINUTES_P456").cast(IntegerType()))
df6 = df6.withColumn("AVG_MINUTES_P789",F.col("AVG_MINUTES_P789").cast(IntegerType()))
df6 = df6.withColumn("AVG_NR_OF_CALLS_P123",F.col("AVG_NR_OF_CALLS_P123").cast(IntegerType()))
df6 = df6.withColumn("AVG_NR_OF_CALLS_P456",F.col("AVG_NR_OF_CALLS_P456").cast(IntegerType()))
df6 = df6.withColumn("AVG_NR_OF_CALLS_P789",F.col("AVG_NR_OF_CALLS_P789").cast(IntegerType()))

In [ ]:

base4 = df6.groupBy("oib").agg(
F.avg("HH_SPEND_ON_NUM").alias("HH_SPEND_ON_NUM"),
F.avg("RATE_HH_SUM_ARPA_M1_AVG_M23").alias("RATE_HH_SUM_ARPA_M1_AVG_M23"),
F.avg("RATE_HH_SUM_ARPA_M2_AVG_M31").alias("RATE_HH_SUM_ARPA_M2_AVG_M31"),
F.avg("RATE_HH_SUM_ARPA_M3_AVG_M12").alias("RATE_HH_SUM_ARPA_M3_AVG_M12"),
F.avg("RATE_HH_SUM_ARPA_M7_AVG_M89").alias("RATE_HH_SUM_ARPA_M7_AVG_M89"),
F.avg("RATE_HH_SUM_ARPA_M8_AVG_M97").alias("RATE_HH_SUM_ARPA_M8_AVG_M97"),
F.avg("RATE_HH_SUM_ARPA_M9_AVG_M78").alias("RATE_HH_SUM_ARPA_M9_AVG_M78"),
F.avg("RATE_SUM_ARPA_M4_AVG_M56").alias("RATE_SUM_ARPA_M4_AVG_M56"),
F.avg("RATE_SUM_ARPA_M5_AVG_M64").alias("RATE_SUM_ARPA_M5_AVG_M64"),
F.avg("RATE_SUM_ARPA_M6_AVG_M45").alias("RATE_SUM_ARPA_M6_AVG_M45"),
F.avg("SUM_ARPA_HH_MIN_1").alias("SUM_ARPA_HH_MIN_1"),
F.avg("SUM_ARPA_HH_MIN_2").alias("SUM_ARPA_HH_MIN_2"),
F.avg("SUM_ARPA_HH_MIN_3").alias("SUM_ARPA_HH_MIN_3"),
F.avg("SUM_ARPA_HH_MIN_4").alias("SUM_ARPA_HH_MIN_4"),
F.avg("SUM_ARPA_HH_MIN_5").alias("SUM_ARPA_HH_MIN_5"),
F.avg("SUM_ARPA_HH_MIN_6").alias("SUM_ARPA_HH_MIN_6"),
F.avg("SUM_ARPA_HH_MIN_7").alias("SUM_ARPA_HH_MIN_7"),
F.avg("SUM_ARPA_HH_MIN_8").alias("SUM_ARPA_HH_MIN_8"),
F.avg("SUM_ARPA_HH_MIN_9").alias("SUM_ARPA_HH_MIN_9"),
F.avg("SUM_ARPA_TOT_HH_MIN_123").alias("SUM_ARPA_TOT_HH_MIN_123"),
F.avg("SUM_ARPA_TOT_HH_MIN_456").alias("SUM_ARPA_TOT_HH_MIN_456"),
F.avg("SUM_ARPA_TOT_HH_MIN_789").alias("SUM_ARPA_TOT_HH_MIN_789"),
F.avg("AVG_TOT_PAY_DIFF_P456_P123").alias("AVG_TOT_PAY_DIFF_P456_P123"),
F.avg("AVG_TOT_PAY_DIFF_P789_P123").alias("AVG_TOT_PAY_DIFF_P789_P123"),
F.avg("AVG_TOT_PAY_DIFF_P789_P456").alias("AVG_TOT_PAY_DIFF_P789_P456"),
F.avg("AVG_TOTAL_PAYAMOUNT_P123").alias("AVG_TOTAL_PAYAMOUNT_P123"),
F.avg("AVG_TOTAL_PAYAMOUNT_P456").alias("AVG_TOTAL_PAYAMOUNT_P456"),
F.avg("AVG_TOTAL_PAYAMOUNT_P789").alias("AVG_TOTAL_PAYAMOUNT_P789"),
F.avg("TOTAL_BILLED_FOR_LAST_3_MONTHS").alias("TOTAL_BILLED_FOR_LAST_3_MONTHS"),
F.avg("TOTAL_BILLED_FOR_LAST_6_MONTHS").alias("TOTAL_BILLED_FOR_LAST_6_MONTHS"),
F.avg("TOTAL_BILLED_FOR_LAST_9_MONTHS").alias("TOTAL_BILLED_FOR_LAST_9_MONTHS"),
F.avg("AVG_MINUTES_P123").alias("AVG_MINUTES_P123"),
F.avg("AVG_MINUTES_P456").alias("AVG_MINUTES_P456"),
F.avg("AVG_MINUTES_P789").alias("AVG_MINUTES_P789"),
F.avg("AVG_NR_OF_CALLS_P123").alias("AVG_NR_OF_CALLS_P123"),
F.avg("AVG_NR_OF_CALLS_P456").alias("AVG_NR_OF_CALLS_P456"),
F.avg("AVG_NR_OF_CALLS_P789").alias("AVG_NR_OF_CALLS_P789"))

In [ ]:
L4 = ["TIMES_IN_CALL_CMP_LAST_3_MNTHS",
"TIMES_IN_CALL_CMP_LAST_6_MNTHS",
"TIMES_IN_CALL_CMP_LAST_9_MNTHS",
"ADM_TOT_CHURN_DIFF_P456_P123",
"ADM_TOT_CHURN_DIFF_P789_P123",
"ADM_TOT_CHURN_DIFF_P789_P456",
"ADM_TOT_CHURN_SUM_P123",
"ADM_TOT_CHURN_SUM_P456",
"ADM_TOT_CHURN_SUM_P789",
"CHA_GROUP_1",
"CHA_GROUP_2",
"CHA_GROUP_3",
"ADSL_SPEED_ACTIVE",
"ADSL_SPEED_TENURE",
"ADSL_TRAFFIC_ACTIVE",
"ADSL_TRAFFIC_TENURE",
"BB_SP_DAY_TILL_CNTRCT_EXP",
"BB_TR_DAY_TILL_CNTRCT_EXP",
"BNDL_DAY_TILL_CNTRCT_EXP",
"BUNDLE_ACTIVE",
"BUNDLE_GROUP",
"BUNDLE_TARIFF_PACKAGE",
"BUNDLE_TENURE",
"EBILL_ACTIVATED_Y_N",
"HALO_ACTIVE",
"HALO_OPTION_NON_STOP",
"HALO_TENURE",
"MAXTV_ACTIVE_MIN",
"OPIS_PONUDE",
"PLAY",
"AGE_GROUP",
"AGE_KUPAC",
"GENDER_KUPAC",
"HH_TYPE",
"HAD_INTERACTION_LAST_3_MONTHS",
"HAD_INTERACTION_LAST_6_MONTHS",
"HAD_INTERACTION_LAST_9_MONTHS",
"ACQ_MAGENTA_IN_LAST_3_MONTHS",
"ACQ_MAGENTA_IN_LAST_6_MONTHS",
"BB_TECHNOLOGY",
"FTTH_FINISHED", 
"MAGENTA_STATUS"    
]

In [ ]:
df7 = df2.select("oib",*L4)

In [ ]:

base5 = df7.groupBy("oib").agg(
F.first("TIMES_IN_CALL_CMP_LAST_3_MNTHS").alias("TIMES_IN_CALL_CMP_LAST_3_MNTHS"),
F.first("TIMES_IN_CALL_CMP_LAST_6_MNTHS").alias("TIMES_IN_CALL_CMP_LAST_6_MNTHS"),
F.first("TIMES_IN_CALL_CMP_LAST_9_MNTHS").alias("TIMES_IN_CALL_CMP_LAST_9_MNTHS"),
F.first("ADM_TOT_CHURN_DIFF_P456_P123").alias("ADM_TOT_CHURN_DIFF_P456_P123"),
F.first("ADM_TOT_CHURN_DIFF_P789_P123").alias("ADM_TOT_CHURN_DIFF_P789_P123"),
F.first("ADM_TOT_CHURN_DIFF_P789_P456").alias("ADM_TOT_CHURN_DIFF_P789_P456"),
F.first("ADM_TOT_CHURN_SUM_P123").alias("ADM_TOT_CHURN_SUM_P123"),
F.first("ADM_TOT_CHURN_SUM_P456").alias("ADM_TOT_CHURN_SUM_P456"),
F.first("ADM_TOT_CHURN_SUM_P789").alias("ADM_TOT_CHURN_SUM_P789"),
F.first("CHA_GROUP_1").alias("CHA_GROUP_1"),
F.first("CHA_GROUP_2").alias("CHA_GROUP_2"),
F.first("CHA_GROUP_3").alias("CHA_GROUP_3"),
F.first("ADSL_SPEED_ACTIVE").alias("ADSL_SPEED_ACTIVE"),
F.first("ADSL_SPEED_TENURE").alias("ADSL_SPEED_TENURE"),
F.first("ADSL_TRAFFIC_ACTIVE").alias("ADSL_TRAFFIC_ACTIVE"),
F.first("ADSL_TRAFFIC_TENURE").alias("ADSL_TRAFFIC_TENURE"),
F.first("BB_SP_DAY_TILL_CNTRCT_EXP").alias("BB_SP_DAY_TILL_CNTRCT_EXP"),
F.first("BB_TR_DAY_TILL_CNTRCT_EXP").alias("BB_TR_DAY_TILL_CNTRCT_EXP"),
F.first("BNDL_DAY_TILL_CNTRCT_EXP").alias("BNDL_DAY_TILL_CNTRCT_EXP"),
F.first("BUNDLE_ACTIVE").alias("BUNDLE_ACTIVE"),
F.first("BUNDLE_GROUP").alias("BUNDLE_GROUP"),
F.first("BUNDLE_TARIFF_PACKAGE").alias("BUNDLE_TARIFF_PACKAGE"),
F.first("BUNDLE_TENURE").alias("BUNDLE_TENURE"),
F.first("EBILL_ACTIVATED_Y_N").alias("EBILL_ACTIVATED_Y_N"),
F.first("HALO_ACTIVE").alias("HALO_ACTIVE"),
F.first("HALO_OPTION_NON_STOP").alias("HALO_OPTION_NON_STOP"),
F.first("HALO_TENURE").alias("HALO_TENURE"),
F.first("MAXTV_ACTIVE_MIN").alias("MAXTV_ACTIVE_MIN"),
F.first("OPIS_PONUDE").alias("OPIS_PONUDE"),
F.first("PLAY").alias("PLAY"),
F.first("AGE_GROUP").alias("AGE_GROUP"),
F.first("AGE_KUPAC").alias("AGE_KUPAC"),
F.first("GENDER_KUPAC").alias("GENDER_KUPAC"),
F.first("HH_TYPE").alias("HH_TYPE"),
F.first("HAD_INTERACTION_LAST_3_MONTHS").alias("HAD_INTERACTION_LAST_3_MONTHS"),
F.first("HAD_INTERACTION_LAST_6_MONTHS").alias("HAD_INTERACTION_LAST_6_MONTHS"),
F.first("HAD_INTERACTION_LAST_9_MONTHS").alias("HAD_INTERACTION_LAST_9_MONTHS"),
F.first("ACQ_MAGENTA_IN_LAST_3_MONTHS").alias("ACQ_MAGENTA_IN_LAST_3_MONTHS"),
F.first("ACQ_MAGENTA_IN_LAST_6_MONTHS").alias("ACQ_MAGENTA_IN_LAST_6_MONTHS"),
F.first("MAGENTA_STATUS").alias("MAGENTA_STATUS"),
F.first("BB_TECHNOLOGY").alias("BB_TECHNOLOGY"),
F.first("FTTH_FINISHED").alias("FTTH_FINISHED"))

In [ ]:
base5.count()

In [ ]:
base5.show()

In [ ]:
L5 = [
    
"HAD_COMPLAINT_LAST_3_MONTHS",
"HAD_COMPLAINT_LAST_6_MONTHS",
"HAD_COMPLAINT_LAST_9_MONTHS",
"FULL_TO_NO_MAGENTA_P2_P1",
"FULL_TO_NO_MAGENTA_P3_P2",
"FULL_TO_PARTIAL_MAGENTA_P2_P1",
"FULL_TO_PARTIAL_MAGENTA_P3_P2",
"LEFT_MAGENTA_IN_LAST_3_MONTHS",
"LEFT_MAGENTA_IN_LAST_6_MONTHS",
"PARTIAL_TO_NO_MAGENTA_P2_P1",
"PARTIAL_TO_NO_MAGENTA_P3_P2"

]

In [ ]:
df8 = df2.select("oib",*L5)

In [ ]:
def scorestr(score):
    s = "\""+score+"\""
    return s
    

In [ ]:
quotes = udf(scorestr)

In [ ]:
df8 = df8.withColumn("HAD_COMPLAINT_LAST_3_MONTHS",F.when((df8.HAD_COMPLAINT_LAST_3_MONTHS == "Y") | (df8.HAD_COMPLAINT_LAST_3_MONTHS == "1"),1).otherwise(0))

df8 = df8.withColumn("HAD_COMPLAINT_LAST_6_MONTHS",F.when((df8.HAD_COMPLAINT_LAST_6_MONTHS == "Y") | (df8.HAD_COMPLAINT_LAST_6_MONTHS == "1"),1).otherwise(0))

df8 = df8.withColumn("HAD_COMPLAINT_LAST_9_MONTHS",F.when((df8.HAD_COMPLAINT_LAST_9_MONTHS == "Y") | (df8.HAD_COMPLAINT_LAST_9_MONTHS == "1"),1).otherwise(0))

df8 = df8.withColumn("FULL_TO_NO_MAGENTA_P2_P1",F.when((df8.FULL_TO_NO_MAGENTA_P2_P1 == "Y") | (df8.FULL_TO_NO_MAGENTA_P2_P1 == "1"),1).otherwise(0))

df8 = df8.withColumn("FULL_TO_NO_MAGENTA_P3_P2",F.when((df8.FULL_TO_NO_MAGENTA_P3_P2 == "Y") | (df8.FULL_TO_NO_MAGENTA_P3_P2 == "1"),1).otherwise(0))

df8 = df8.withColumn("FULL_TO_PARTIAL_MAGENTA_P2_P1",F.when((df8.FULL_TO_PARTIAL_MAGENTA_P2_P1 == "Y") | (df8.FULL_TO_PARTIAL_MAGENTA_P2_P1 == "1"),1).otherwise(0))

df8 = df8.withColumn("FULL_TO_PARTIAL_MAGENTA_P3_P2",F.when((df8.FULL_TO_PARTIAL_MAGENTA_P3_P2 == "Y") | (df8.FULL_TO_PARTIAL_MAGENTA_P3_P2 == "1"),1).otherwise(0))

df8 = df8.withColumn("LEFT_MAGENTA_IN_LAST_3_MONTHS",F.when((df8.LEFT_MAGENTA_IN_LAST_3_MONTHS == "Y") | (df8.LEFT_MAGENTA_IN_LAST_3_MONTHS == "1"),1).otherwise(0))

df8 = df8.withColumn("LEFT_MAGENTA_IN_LAST_6_MONTHS",F.when((df8.LEFT_MAGENTA_IN_LAST_6_MONTHS == "Y") | (df8.LEFT_MAGENTA_IN_LAST_6_MONTHS == "1"),1).otherwise(0))

df8 = df8.withColumn("PARTIAL_TO_NO_MAGENTA_P2_P1",F.when((df8.PARTIAL_TO_NO_MAGENTA_P2_P1 == "Y") | (df8.PARTIAL_TO_NO_MAGENTA_P2_P1 == "1"),1).otherwise(0))

df8 = df8.withColumn("PARTIAL_TO_NO_MAGENTA_P3_P2",F.when((df8.PARTIAL_TO_NO_MAGENTA_P3_P2 == "Y") | (df8.PARTIAL_TO_NO_MAGENTA_P3_P2 == "1"),1).otherwise(0))


In [ ]:
# df8 = df8.withColumn("HAD_COMPLAINT_LAST_3_MONTHS",quotes("HAD_COMPLAINT_LAST_3_MONTHS"))
# df8 = df8.withColumn("HAD_COMPLAINT_LAST_6_MONTHS",quotes("HAD_COMPLAINT_LAST_6_MONTHS"))
# df8 = df8.withColumn("HAD_COMPLAINT_LAST_9_MONTHS",quotes("HAD_COMPLAINT_LAST_9_MONTHS"))
# df8 = df8.withColumn("FULL_TO_NO_MAGENTA_P2_P1",quotes("FULL_TO_NO_MAGENTA_P2_P1"))
# df8 = df8.withColumn("FULL_TO_NO_MAGENTA_P3_P2",quotes("FULL_TO_NO_MAGENTA_P3_P2"))
# df8 = df8.withColumn("FULL_TO_PARTIAL_MAGENTA_P2_P1",quotes("FULL_TO_PARTIAL_MAGENTA_P2_P1"))
# df8 = df8.withColumn("FULL_TO_PARTIAL_MAGENTA_P3_P2",quotes("FULL_TO_PARTIAL_MAGENTA_P3_P2"))
# df8 = df8.withColumn("LEFT_MAGENTA_IN_LAST_3_MONTHS",quotes("LEFT_MAGENTA_IN_LAST_3_MONTHS"))
# df8 = df8.withColumn("LEFT_MAGENTA_IN_LAST_6_MONTHS",quotes("LEFT_MAGENTA_IN_LAST_6_MONTHS"))
# df8 = df8.withColumn("PARTIAL_TO_NO_MAGENTA_P2_P1",quotes("PARTIAL_TO_NO_MAGENTA_P2_P1"))
# df8 = df8.withColumn("PARTIAL_TO_NO_MAGENTA_P3_P2",quotes("PARTIAL_TO_NO_MAGENTA_P3_P2"))


In [ ]:
def YESorNO(data_list):
    li = np.array(data_list)
    sum1 = np.sum(li)
    if sum1 >0:
        return "Y"
    else:
        return "N"

In [ ]:
YESorNO = udf(YESorNO)

In [ ]:
base6 = df8.groupBy("oib").agg(F.collect_list("HAD_COMPLAINT_LAST_3_MONTHS").alias("HAD_COMPLAINT_LAST_3_MONTHS"),F.collect_list("HAD_COMPLAINT_LAST_6_MONTHS").alias("HAD_COMPLAINT_LAST_6_MONTHS"),
F.collect_list("HAD_COMPLAINT_LAST_9_MONTHS").alias("HAD_COMPLAINT_LAST_9_MONTHS"),
F.collect_list("FULL_TO_NO_MAGENTA_P2_P1").alias("FULL_TO_NO_MAGENTA_P2_P1"),
F.collect_list("FULL_TO_NO_MAGENTA_P3_P2").alias("FULL_TO_NO_MAGENTA_P3_P2"),
F.collect_list("FULL_TO_PARTIAL_MAGENTA_P2_P1").alias("FULL_TO_PARTIAL_MAGENTA_P2_P1"),
F.collect_list("FULL_TO_PARTIAL_MAGENTA_P3_P2").alias("FULL_TO_PARTIAL_MAGENTA_P3_P2"),
F.collect_list("LEFT_MAGENTA_IN_LAST_3_MONTHS").alias("LEFT_MAGENTA_IN_LAST_3_MONTHS"),
F.collect_list("LEFT_MAGENTA_IN_LAST_6_MONTHS").alias("LEFT_MAGENTA_IN_LAST_6_MONTHS"),

F.collect_list("PARTIAL_TO_NO_MAGENTA_P2_P1").alias("PARTIAL_TO_NO_MAGENTA_P2_P1"),
F.collect_list("PARTIAL_TO_NO_MAGENTA_P3_P2").alias("PARTIAL_TO_NO_MAGENTA_P3_P2")).withColumn("HAD_COMPLAINT_LAST_3_MONTHS",YESorNO("HAD_COMPLAINT_LAST_3_MONTHS")).withColumn("HAD_COMPLAINT_LAST_6_MONTHS",YESorNO("HAD_COMPLAINT_LAST_6_MONTHS")).withColumn("HAD_COMPLAINT_LAST_9_MONTHS",YESorNO("HAD_COMPLAINT_LAST_9_MONTHS")).withColumn("FULL_TO_NO_MAGENTA_P2_P1",YESorNO("FULL_TO_NO_MAGENTA_P2_P1")).withColumn("FULL_TO_NO_MAGENTA_P3_P2",YESorNO("FULL_TO_NO_MAGENTA_P3_P2")).withColumn("FULL_TO_PARTIAL_MAGENTA_P2_P1",YESorNO("FULL_TO_PARTIAL_MAGENTA_P2_P1")).withColumn("FULL_TO_PARTIAL_MAGENTA_P3_P2",YESorNO("FULL_TO_PARTIAL_MAGENTA_P3_P2")).withColumn("LEFT_MAGENTA_IN_LAST_3_MONTHS",YESorNO("LEFT_MAGENTA_IN_LAST_3_MONTHS")).withColumn("LEFT_MAGENTA_IN_LAST_6_MONTHS",YESorNO("LEFT_MAGENTA_IN_LAST_6_MONTHS")).withColumn("PARTIAL_TO_NO_MAGENTA_P2_P1",YESorNO("PARTIAL_TO_NO_MAGENTA_P2_P1")).withColumn("PARTIAL_TO_NO_MAGENTA_P3_P2",YESorNO("PARTIAL_TO_NO_MAGENTA_P3_P2"))

In [ ]:
base1.count()

In [ ]:
base2.count()

In [ ]:
base3.count()

In [ ]:
base4.count()

In [ ]:
base5.count()

In [ ]:
base6.count()

In [ ]:
base = base1.join(base2,["oib"],"left")
base = base.join(base3,["oib"],"left")
base = base.join(base4,["oib"],"left")
base = base.join(base5,["oib"],"left")
base = base.join(base6,["oib"],"left")

In [ ]:
base.count()

In [ ]:
fix_scoring_202007

In [ ]:
base.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.jan_31_fix_scoring_202012 as select * from myTemp11")


In [ ]:
tele_id = dataImport("sbx_adhoc_ds"," subscriber_id_connection_oib_20210222")

In [ ]:
for i in tele_id.columns:
    tele_id = tele_id.withColumn(i, regexp_replace(i, '"', ''))

In [ ]:
tele_id.select("period").distinct().show()

In [ ]:
tele_id = tele_id.filter(F.col("period") == "202101")

In [ ]:
tele_id = tele_id.filter(F.col("oib") !="")

In [ ]:
tele_id = tele_id.drop_duplicates(["telephone_number_id","oib"])

In [ ]:
tele_id.count()

In [ ]:
mob_09 = dataImport("sbx_adhoc_ds"," mob_scoring_eop202101_20210222")

In [ ]:
mob_w = dataImport("sbx_adhoc_ds"," mob_subscribers_202101_w_consent")

In [ ]:
for i in mob_09.columns:
    mob_09 = mob_09.withColumn(i, regexp_replace(i, '"', ''))

In [ ]:
mob_w1 = mob_w.drop_duplicates(["telephone_number_id"])

In [ ]:
mob_09 = mob_09.join(mob_w1,["telephone_number_id"],"inner")

In [ ]:
mob_09 = mob_09.drop("period")

In [ ]:
mob_09.count()

In [ ]:
for i in mob_09.columns:
    mob_09 = mob_09.withColumn(i, regexp_replace(i, '"', ''))

In [ ]:
mob_09 = mob_09.drop_duplicates(["telephone_number_id"])

In [ ]:
len(mob_09.columns)

In [ ]:
tele_id  = tele_id.select("telephone_number_id")

In [ ]:
mob_09_tele = mob_09.join(tele_id,["telephone_number_id"],"inner")

In [ ]:
L1 = [
    
    'QTY_COMPLAINT_PMIN2',
'QTY_COMPLAINT_PMIN3',
'QTY_COMPLAINT_PMIN4',
'RATE_TMHR_CALLS_PMIN2',
'RATE_BONBON_CALLS_PMIN2',
'RATE_VIP_CALLS_PMIN2',
'RATE_TELE2_CALLS_PMIN2',
'RATE_TOMATO_CALLS_PMIN2',
'RATE_INTRNTIONAL_CALLS_PMIN2',
'RATE_FIXED_CALLS_PMIN2',
'RATE_VAS_CALLS_PMIN2',
'RATE_ROAMING_CALLS_PMIN2',
'RATE_TMHR_CALLS_PMIN3',
'RATE_BONBON_CALLS_PMIN3',
'RATE_VIP_CALLS_PMIN3',
'RATE_TELE2_CALLS_PMIN3',
'RATE_TOMATO_CALLS_PMIN3',
'RATE_INTRNTIONAL_CALLS_PMIN3',
'RATE_FIXED_CALLS_PMIN3',
'RATE_VAS_CALLS_PMIN3',
'RATE_ROAMING_CALLS_PMIN3',
'RATE_TMHR_CALLS_PMIN4',
'RATE_BONBON_CALLS_PMIN4',
'RATE_VIP_CALLS_PMIN4',
'RATE_TELE2_CALLS_PMIN4',
'RATE_TOMATO_CALLS_PMIN4',
'RATE_INTRNTIONAL_CALLS_PMIN4',
'RATE_FIXED_CALLS_PMIN4',
'RATE_VAS_CALLS_PMIN4',
'RATE_ROAMING_CALLS_PMIN4',
'RATE_TMHR_MIN_PMIN2',
'RATE_BONBON_MIN_PMIN2',
'RATE_VIP_MIN_PMIN2',
'RATE_TELE2_MIN_PMIN2',
'RATE_TOMATO_MIN_PMIN2',
'RATE_INTRNTIONAL_MIN_PMIN2',
'RATE_FIXED_MIN_PMIN2',
'RATE_VAS_MIN_PMIN2',
'RATE_ROAMING_MIN_PMIN2',
'RATE_TMHR_MIN_PMIN3',
'RATE_BONBON_MIN_PMIN3',
'RATE_VIP_MIN_PMIN3',
'RATE_TELE2_MIN_PMIN3',
'RATE_TOMATO_MIN_PMIN3',
'RATE_INTRNTIONAL_MIN_PMIN3',
'RATE_FIXED_MIN_PMIN3',
'RATE_VAS_MIN_PMIN3',
'RATE_ROAMING_MIN_PMIN3',
'RATE_TMHR_MIN_PMIN4',
'RATE_BONBON_MIN_PMIN4',
'RATE_VIP_MIN_PMIN4',
'RATE_TELE2_MIN_PMIN4',
'RATE_TOMATO_MIN_PMIN4',
'RATE_INTRNTIONAL_MIN_PMIN4',
'RATE_FIXED_MIN_PMIN4',
'RATE_VAS_MIN_PMIN4',
'RATE_ROAMING_MIN_PMIN4'
]

In [ ]:
L2 = [
    
'QTY_FIX_INST_IN_HH_PMIN2',
'QTY_FIX_INST_IN_HH_PMIN3',
'QTY_FIX_INST_IN_HH_PMIN4',
'QTY_MOB_IN_HH_PMIN2',
'QTY_MOB_IN_HH_PMIN3',
'QTY_MOB_IN_HH_PMIN4'

]

In [ ]:
mob_09_tele_max = mob_09_tele.select("oib",*L1,*L2)

In [ ]:
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_COMPLAINT_PMIN2",F.col("QTY_COMPLAINT_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_COMPLAINT_PMIN3",F.col("QTY_COMPLAINT_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_COMPLAINT_PMIN4",F.col("QTY_COMPLAINT_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TMHR_CALLS_PMIN2",F.col("RATE_TMHR_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_BONBON_CALLS_PMIN2",F.col("RATE_BONBON_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VIP_CALLS_PMIN2",F.col("RATE_VIP_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TELE2_CALLS_PMIN2",F.col("RATE_TELE2_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TOMATO_CALLS_PMIN2",F.col("RATE_TOMATO_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_INTRNTIONAL_CALLS_PMIN2",F.col("RATE_INTRNTIONAL_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_FIXED_CALLS_PMIN2",F.col("RATE_FIXED_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VAS_CALLS_PMIN2",F.col("RATE_VAS_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_ROAMING_CALLS_PMIN2",F.col("RATE_ROAMING_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TMHR_CALLS_PMIN3",F.col("RATE_TMHR_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_BONBON_CALLS_PMIN3",F.col("RATE_BONBON_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VIP_CALLS_PMIN3",F.col("RATE_VIP_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TELE2_CALLS_PMIN3",F.col("RATE_TELE2_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TOMATO_CALLS_PMIN3",F.col("RATE_TOMATO_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_INTRNTIONAL_CALLS_PMIN3",F.col("RATE_INTRNTIONAL_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_FIXED_CALLS_PMIN3",F.col("RATE_FIXED_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VAS_CALLS_PMIN3",F.col("RATE_VAS_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_ROAMING_CALLS_PMIN3",F.col("RATE_ROAMING_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TMHR_CALLS_PMIN4",F.col("RATE_TMHR_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_BONBON_CALLS_PMIN4",F.col("RATE_BONBON_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VIP_CALLS_PMIN4",F.col("RATE_VIP_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TELE2_CALLS_PMIN4",F.col("RATE_TELE2_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TOMATO_CALLS_PMIN4",F.col("RATE_TOMATO_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_INTRNTIONAL_CALLS_PMIN4",F.col("RATE_INTRNTIONAL_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_FIXED_CALLS_PMIN4",F.col("RATE_FIXED_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VAS_CALLS_PMIN4",F.col("RATE_VAS_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_ROAMING_CALLS_PMIN4",F.col("RATE_ROAMING_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TMHR_MIN_PMIN2",F.col("RATE_TMHR_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_BONBON_MIN_PMIN2",F.col("RATE_BONBON_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VIP_MIN_PMIN2",F.col("RATE_VIP_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TELE2_MIN_PMIN2",F.col("RATE_TELE2_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TOMATO_MIN_PMIN2",F.col("RATE_TOMATO_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_INTRNTIONAL_MIN_PMIN2",F.col("RATE_INTRNTIONAL_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_FIXED_MIN_PMIN2",F.col("RATE_FIXED_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VAS_MIN_PMIN2",F.col("RATE_VAS_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_ROAMING_MIN_PMIN2",F.col("RATE_ROAMING_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TMHR_MIN_PMIN3",F.col("RATE_TMHR_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_BONBON_MIN_PMIN3",F.col("RATE_BONBON_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VIP_MIN_PMIN3",F.col("RATE_VIP_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TELE2_MIN_PMIN3",F.col("RATE_TELE2_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TOMATO_MIN_PMIN3",F.col("RATE_TOMATO_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_INTRNTIONAL_MIN_PMIN3",F.col("RATE_INTRNTIONAL_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_FIXED_MIN_PMIN3",F.col("RATE_FIXED_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VAS_MIN_PMIN3",F.col("RATE_VAS_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_ROAMING_MIN_PMIN3",F.col("RATE_ROAMING_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TMHR_MIN_PMIN4",F.col("RATE_TMHR_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_BONBON_MIN_PMIN4",F.col("RATE_BONBON_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VIP_MIN_PMIN4",F.col("RATE_VIP_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TELE2_MIN_PMIN4",F.col("RATE_TELE2_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_TOMATO_MIN_PMIN4",F.col("RATE_TOMATO_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_INTRNTIONAL_MIN_PMIN4",F.col("RATE_INTRNTIONAL_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_FIXED_MIN_PMIN4",F.col("RATE_FIXED_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_VAS_MIN_PMIN4",F.col("RATE_VAS_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("RATE_ROAMING_MIN_PMIN4",F.col("RATE_ROAMING_MIN_PMIN4").cast(IntegerType()))
#mob_09_tele_max = mob_09_tele_max.withColumn("MONTHS_TILL_CONTRACT_END_PMIN2",F.col("MONTHS_TILL_CONTRACT_END_PMIN2").cast(IntegerType()))


mob_09_tele_max = mob_09_tele_max.withColumn("QTY_FIX_INST_IN_HH_PMIN2",F.col("QTY_FIX_INST_IN_HH_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_FIX_INST_IN_HH_PMIN3",F.col("QTY_FIX_INST_IN_HH_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_FIX_INST_IN_HH_PMIN4",F.col("QTY_FIX_INST_IN_HH_PMIN4").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_MOB_IN_HH_PMIN2",F.col("QTY_MOB_IN_HH_PMIN2").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_MOB_IN_HH_PMIN3",F.col("QTY_MOB_IN_HH_PMIN3").cast(IntegerType()))
mob_09_tele_max = mob_09_tele_max.withColumn("QTY_MOB_IN_HH_PMIN4",F.col("QTY_MOB_IN_HH_PMIN4").cast(IntegerType()))

In [ ]:
mob_09_tele_max = mob_09_tele_max.groupBy("oib").agg(
    F.sum("QTY_FIX_INST_IN_HH_PMIN2").alias("QTY_FIX_INST_IN_HH_PMIN2"),
    F.sum("QTY_FIX_INST_IN_HH_PMIN3").alias("QTY_FIX_INST_IN_HH_PMIN3"),
    F.sum("QTY_FIX_INST_IN_HH_PMIN4").alias("QTY_FIX_INST_IN_HH_PMIN4"),
    F.sum("QTY_MOB_IN_HH_PMIN2").alias("QTY_MOB_IN_HH_PMIN2"),
    F.sum("QTY_MOB_IN_HH_PMIN3").alias("QTY_MOB_IN_HH_PMIN3"),
    F.sum("QTY_MOB_IN_HH_PMIN4").alias("QTY_MOB_IN_HH_PMIN4"),
    
#F.min("MONTHS_TILL_CONTRACT_END_PMIN2").alias("MONTHS_TILL_CONTRACT_END_PMIN2"),
F.max("QTY_COMPLAINT_PMIN2").alias("QTY_COMPLAINT_PMIN2"),
F.max("QTY_COMPLAINT_PMIN3").alias("QTY_COMPLAINT_PMIN3"),
F.max("QTY_COMPLAINT_PMIN4").alias("QTY_COMPLAINT_PMIN4"),
F.max("RATE_TMHR_CALLS_PMIN2").alias("RATE_TMHR_CALLS_PMIN2"),
F.max("RATE_BONBON_CALLS_PMIN2").alias("RATE_BONBON_CALLS_PMIN2"),
F.max("RATE_VIP_CALLS_PMIN2").alias("RATE_VIP_CALLS_PMIN2"),
F.max("RATE_TELE2_CALLS_PMIN2").alias("RATE_TELE2_CALLS_PMIN2"),
F.max("RATE_TOMATO_CALLS_PMIN2").alias("RATE_TOMATO_CALLS_PMIN2"),
F.max("RATE_INTRNTIONAL_CALLS_PMIN2").alias("RATE_INTRNTIONAL_CALLS_PMIN2"),
F.max("RATE_FIXED_CALLS_PMIN2").alias("RATE_FIXED_CALLS_PMIN2"),
F.max("RATE_VAS_CALLS_PMIN2").alias("RATE_VAS_CALLS_PMIN2"),
F.max("RATE_ROAMING_CALLS_PMIN2").alias("RATE_ROAMING_CALLS_PMIN2"),
F.max("RATE_TMHR_CALLS_PMIN3").alias("RATE_TMHR_CALLS_PMIN3"),
F.max("RATE_BONBON_CALLS_PMIN3").alias("RATE_BONBON_CALLS_PMIN3"),
F.max("RATE_VIP_CALLS_PMIN3").alias("RATE_VIP_CALLS_PMIN3"),
F.max("RATE_TELE2_CALLS_PMIN3").alias("RATE_TELE2_CALLS_PMIN3"),
F.max("RATE_TOMATO_CALLS_PMIN3").alias("RATE_TOMATO_CALLS_PMIN3"),
F.max("RATE_INTRNTIONAL_CALLS_PMIN3").alias("RATE_INTRNTIONAL_CALLS_PMIN3"),
F.max("RATE_FIXED_CALLS_PMIN3").alias("RATE_FIXED_CALLS_PMIN3"),
F.max("RATE_VAS_CALLS_PMIN3").alias("RATE_VAS_CALLS_PMIN3"),
F.max("RATE_ROAMING_CALLS_PMIN3").alias("RATE_ROAMING_CALLS_PMIN3"),
F.max("RATE_TMHR_CALLS_PMIN4").alias("RATE_TMHR_CALLS_PMIN4"),
F.max("RATE_BONBON_CALLS_PMIN4").alias("RATE_BONBON_CALLS_PMIN4"),
F.max("RATE_VIP_CALLS_PMIN4").alias("RATE_VIP_CALLS_PMIN4"),
F.max("RATE_TELE2_CALLS_PMIN4").alias("RATE_TELE2_CALLS_PMIN4"),
F.max("RATE_TOMATO_CALLS_PMIN4").alias("RATE_TOMATO_CALLS_PMIN4"),
F.max("RATE_INTRNTIONAL_CALLS_PMIN4").alias("RATE_INTRNTIONAL_CALLS_PMIN4"),
F.max("RATE_FIXED_CALLS_PMIN4").alias("RATE_FIXED_CALLS_PMIN4"),
F.max("RATE_VAS_CALLS_PMIN4").alias("RATE_VAS_CALLS_PMIN4"),
F.max("RATE_ROAMING_CALLS_PMIN4").alias("RATE_ROAMING_CALLS_PMIN4"),
F.max("RATE_TMHR_MIN_PMIN2").alias("RATE_TMHR_MIN_PMIN2"),
F.max("RATE_BONBON_MIN_PMIN2").alias("RATE_BONBON_MIN_PMIN2"),
F.max("RATE_VIP_MIN_PMIN2").alias("RATE_VIP_MIN_PMIN2"),
F.max("RATE_TELE2_MIN_PMIN2").alias("RATE_TELE2_MIN_PMIN2"),
F.max("RATE_TOMATO_MIN_PMIN2").alias("RATE_TOMATO_MIN_PMIN2"),
F.max("RATE_INTRNTIONAL_MIN_PMIN2").alias("RATE_INTRNTIONAL_MIN_PMIN2"),
F.max("RATE_FIXED_MIN_PMIN2").alias("RATE_FIXED_MIN_PMIN2"),
F.max("RATE_VAS_MIN_PMIN2").alias("RATE_VAS_MIN_PMIN2"),
F.max("RATE_ROAMING_MIN_PMIN2").alias("RATE_ROAMING_MIN_PMIN2"),
F.max("RATE_TMHR_MIN_PMIN3").alias("RATE_TMHR_MIN_PMIN3"),
F.max("RATE_BONBON_MIN_PMIN3").alias("RATE_BONBON_MIN_PMIN3"),
F.max("RATE_VIP_MIN_PMIN3").alias("RATE_VIP_MIN_PMIN3"),
F.max("RATE_TELE2_MIN_PMIN3").alias("RATE_TELE2_MIN_PMIN3"),
F.max("RATE_TOMATO_MIN_PMIN3").alias("RATE_TOMATO_MIN_PMIN3"),
F.max("RATE_INTRNTIONAL_MIN_PMIN3").alias("RATE_INTRNTIONAL_MIN_PMIN3"),
F.max("RATE_FIXED_MIN_PMIN3").alias("RATE_FIXED_MIN_PMIN3"),
F.max("RATE_VAS_MIN_PMIN3").alias("RATE_VAS_MIN_PMIN3"),
F.max("RATE_ROAMING_MIN_PMIN3").alias("RATE_ROAMING_MIN_PMIN3"),
F.max("RATE_TMHR_MIN_PMIN4").alias("RATE_TMHR_MIN_PMIN4"),
F.max("RATE_BONBON_MIN_PMIN4").alias("RATE_BONBON_MIN_PMIN4"),
F.max("RATE_VIP_MIN_PMIN4").alias("RATE_VIP_MIN_PMIN4"),
F.max("RATE_TELE2_MIN_PMIN4").alias("RATE_TELE2_MIN_PMIN4"),
F.max("RATE_TOMATO_MIN_PMIN4").alias("RATE_TOMATO_MIN_PMIN4"),
F.max("RATE_INTRNTIONAL_MIN_PMIN4").alias("RATE_INTRNTIONAL_MIN_PMIN4"),
F.max("RATE_FIXED_MIN_PMIN4").alias("RATE_FIXED_MIN_PMIN4"),
F.max("RATE_VAS_MIN_PMIN4").alias("RATE_VAS_MIN_PMIN4"),
F.max("RATE_ROAMING_MIN_PMIN4").alias("RATE_ROAMING_MIN_PMIN4"))

In [ ]:
mob_09_tele_max.count()

In [ ]:
L3 = [
    
    'VOICE_ALL_CALLS_PMIN2',
'VOICE_ALL_CALLS_PMIN3',
'VOICE_ALL_CALLS_PMIN4',
'VOICE_TMHR_CALLS_PMIN2',
'VOICE_TMHR_CALLS_PMIN3',
'VOICE_TMHR_CALLS_PMIN4',
'VOICE_BONBON_CALLS_PMIN2',
'VOICE_BONBON_CALLS_PMIN3',
'VOICE_BONBON_CALLS_PMIN4',
'VOICE_VIP_CALLS_PMIN2',
'VOICE_VIP_CALLS_PMIN3',
'VOICE_VIP_CALLS_PMIN4',
'VOICE_TELE2_CALLS_PMIN2',
'VOICE_TELE2_CALLS_PMIN3',
'VOICE_TELE2_CALLS_PMIN4',
'VOICE_TOMATO_CALLS_PMIN2',
'VOICE_TOMATO_CALLS_PMIN3',
'VOICE_TOMATO_CALLS_PMIN4',
'VOICE_INTRNTIONAL_CALLS_PMIN2',
'VOICE_INTRNTIONAL_CALLS_PMIN3',
'VOICE_INTRNTIONAL_CALLS_PMIN4',
'VOICE_FIXED_CALLS_MIN2',
'VOICE_FIXED_CALLS_MIN3',
'VOICE_FIXED_CALLS_MIN4',
'VOICE_VAS_CALLS_PMIN2',
'VOICE_VAS_CALLS_PMIN3',
'VOICE_VAS_CALLS_PMIN4',
'VOICE_ROAMING_CALLS_PMIN2',
'VOICE_ROAMING_CALLS_PMIN3',
'VOICE_ROAMING_CALLS_PMIN4',
'VOICE_ALL_MIN_PMIN2',
'VOICE_ALL_MIN_PMIN3',
'VOICE_ALL_MIN_PMIN4',
'VOICE_TMHR_MIN_PMIN2',
'VOICE_TMHR_MIN_PMIN3',
'VOICE_TMHR_MIN_PMIN4',



'VOICE_VIP_MIN_PMIN2',
'VOICE_VIP_MIN_PMIN3',
'VOICE_VIP_MIN_PMIN4',
'VOICE_TELE2_MIN_PMIN2',
'VOICE_TELE2_MIN_PMIN3',
'VOICE_TELE2_MIN_PMIN4',
'VOICE_TOMATO_MIN_PMIN2',
'VOICE_TOMATO_MIN_PMIN3',
'VOICE_TOMATO_MIN_PMIN4',
'VOICE_INTERNATIONAL_MIN_PMIN2',
'VOICE_INTERNATIONAL_MIN_PMIN3',
'VOICE_INTERNATIONAL_MIN_PMIN4',
'VOICE_FIXED_MIN_PMIN2',
'VOICE_FIXED_MIN_PMIN3',
'VOICE_FIXED_MIN_PMIN4',
'VOICE_VAS_MIN_PMIN2',
'VOICE_VAS_MIN_PMIN3',
'VOICE_VAS_MIN_PMIN4',
'VOICE_ROAMING_MIN_PMIN2',
'VOICE_ROAMING_MIN_PMIN3',
'VOICE_ROAMING_MIN_PMIN4',
'SMS_ALL_PMIN2',
'SMS_ALL_PMIN3',
'SMS_ALL_PMIN4',
'SMS_TMHR_PMIN2',
'SMS_TMHR_PMIN3',
'SMS_TMHR_PMIN4',
'SMS_BONBON_PMIN2',
'SMS_BONBON_PMIN3',
'SMS_BONBON_PMIN4',
'SMS_VIP_PMIN2',
'SMS_VIP_PMIN3',
'SMS_VIP_PMIN4',
'SMS_TELE2_PMIN2',
'SMS_TELE2_PMIN3',
'SMS_TELE2_PMIN4',
'SMS_TOMATO_PMIN2',
'SMS_TOMATO_PMIN3',
'SMS_TOMATO_PMIN4',
'SMS_INTERNATIONAL_PMIN2',
'SMS_INTERNATIONAL_PMIN3',
'SMS_INTERNATIONAL_PMIN4',
'SMS_VAS_PMIN2',
'SMS_VAS_PMIN3',
'SMS_VAS_PMIN4',
'SMS_ROAMING_PMIN2',
'SMS_ROAMING_PMIN3',
'SMS_ROAMING_PMIN4',
'DATA_NATIONAL_VOLUME_MB_PMIN2',
'DATA_NATIONAL_VOLUME_MB_PMIN3',
'DATA_NATIONAL_VOLUME_MB_PMIN4',
'DATA_NATIONAL_SESSIONS_PMIN2',
'DATA_NATIONAL_SESSIONS_PMIN3',
'DATA_NATIONAL_SESSIONS_PMIN4',
'DATA_ROAMING_VOLUME_MB_PMIN2',
'DATA_ROAMING_VOLUME_MB_PMIN3',
'DATA_ROAMING_VOLUME_MB_PMIN4',
'DATA_ROAMING_SESSIONS_PMIN2',
'DATA_ROAMING_SESSIONS_PMIN3',
'DATA_ROAMING_SESSIONS_PMIN4',
'INT_CALL_IN_ADMIN_ACT_PMIN2',
'INT_CALL_IN_ADMIN_ACT_PMIN3',
'INT_CALL_IN_ADMIN_ACT_PMIN4',
'INT_CALL_IN_HRC_PMIN2',
'INT_CALL_IN_HRC_PMIN3',
'INT_CALL_IN_HRC_PMIN4',
'INT_CALL_IN_ICCA_TIM_PMIN2',
'INT_CALL_IN_ICCA_TIM_PMIN3',
'INT_CALL_IN_ICCA_TIM_PMIN4',
'INT_CALL_IN_INFO_PMIN2',
'INT_CALL_IN_INFO_PMIN3',
'INT_CALL_IN_INFO_PMIN4',
'INT_CALL_IN_INTERACTION_PMIN2',
'INT_CALL_IN_INTERACTION_PMIN3',
'INT_CALL_IN_INTERACTION_PMIN4',
'INT_CALL_IN_USLUGE_KAMP_PMIN2',
'INT_CALL_IN_USLUGE_KAMP_PMIN3',
'INT_CALL_IN_USLUGE_KAMP_PMIN4',
'INT_CALL_IN_OSTALO_PMIN2',
'INT_CALL_IN_OSTALO_PMIN3',
'INT_CALL_IN_OSTALO_PMIN4',
'INT_CALL_IN_RACUNI_PMIN2',
'INT_CALL_IN_RACUNI_PMIN3',
'INT_CALL_IN_RACUNI_PMIN4',
'INT_CALL_IN_POS_KOR_PMIN2',
'INT_CALL_IN_POS_KOR_PMIN3',
'INT_CALL_IN_POS_KOR_PMIN4',
'INT_CALL_IN_PRIGOVOR_PMIN2',
'INT_CALL_IN_PRIGOVOR_PMIN3',
'INT_CALL_IN_PRIGOVOR_PMIN4',
'INT_CALL_IN_UGOVOR_PMIN2',
'INT_CALL_IN_UGOVOR_PMIN3',
'INT_CALL_IN_UGOVOR_PMIN4',
'INT_CALL_IN_SAVEDESK_PMIN2',
'INT_CALL_IN_SAVEDESK_PMIN3',
'INT_CALL_IN_SAVEDESK_PMIN4',
'INT_CALL_IN_TEHNICKA_PMIN2',
'INT_CALL_IN_TEHNICKA_PMIN3',
'INT_CALL_IN_TEHNICKA_PMIN4',
'INT_CALL_IN_TELESALES_PMIN2',
'INT_CALL_IN_TELESALES_PMIN3',
'INT_CALL_IN_TELESALES_PMIN4',
'INT_CALL_IN_TRIM_TIM_PMIN2',
'INT_CALL_IN_TRIM_TIM_PMIN3',
'INT_CALL_IN_TRIM_TIM_PMIN4',
'INT_CALL_IN_X7_PMIN2',
'INT_CALL_IN_X7_PMIN3',
'INT_CALL_IN_X7_PMIN4',
'QTY_DAYS_INTERACTING_PMIN2',
'QTY_DAYS_INTERACTING_PMIN3',
'QTY_DAYS_INTERACTING_PMIN4',
'INVOICED_PMIN2',
'INVOICED_PMIN3',
'INVOICED_PMIN4',
'REVENUE_PMIN2',
'REVENUE_PMIN3',
'REVENUE_PMIN4',
'ROAMING_AMOUNT_PMIN2',
'ROAMING_AMOUNT_PMIN3',
'ROAMING_AMOUNT_PMIN4',
'REVENUE_SUBSCRIPTION_PMIN2',
'REVENUE_SUBSCRIPTION_PMIN3',
'REVENUE_SUBSCRIPTION_PMIN4',
'REVENUE_VOICE_PMIN2',
'REVENUE_VOICE_PMIN3',
'REVENUE_VOICE_PMIN4',
'REVENUE_SMS_PMIN2',
'REVENUE_SMS_PMIN3',
'REVENUE_SMS_PMIN4',
'REVENUE_DATA_NATIONAL_PMIN2',
'REVENUE_DATA_NATIONAL_PMIN3',
'REVENUE_DATA_NATIONAL_PMIN4',
'REVENUE_DATA_ROAMING_PMIN2',
'REVENUE_DATA_ROAMING_PMIN3',
'REVENUE_DATA_ROAMING_PMIN4',
'REVENUE_VOICE_VAS_PMIN2',
'REVENUE_VOICE_VAS_PMIN3',
'REVENUE_VOICE_VAS_PMIN4',
'REVENUE_SMS_VAS_PMIN2',
'REVENUE_SMS_VAS_PMIN3',
'REVENUE_SMS_VAS_PMIN4',
'REVENUE_GSM_LATE_FEE_PMIN2',
'REVENUE_GSM_LATE_FEE_PMIN3',
'REVENUE_GSM_LATE_FEE_PMIN4',
'REVENUE_GSM_OTHER_PMIN2',
'REVENUE_GSM_OTHER_PMIN3',
'REVENUE_GSM_OTHER_PMIN4',
'COST_VOICE_ONNET_PMIN2',
'COST_VOICE_ONNET_PMIN3',
'COST_VOICE_ONNET_PMIN4',
'COST_VOICE_OFFNET_PMIN2',
'COST_VOICE_OFFNET_PMIN3',
'COST_VOICE_OFFNET_PMIN4',
'COST_VOICE_VIP_PMIN2',
'COST_VOICE_VIP_PMIN3',
'COST_VOICE_VIP_PMIN4',
'COST_VOICE_TELE2_PMIN2',
'COST_VOICE_TELE2_PMIN3',
'COST_VOICE_TELE2_PMIN4',
'COST_VOICE_TOMATO_PMIN2',
'COST_VOICE_TOMATO_PMIN3',
'COST_VOICE_TOMATO_PMIN4',
'COST_VOICE_BONBON_PMIN2',
'COST_VOICE_BONBON_PMIN3',
'COST_VOICE_BONBON_PMIN4',
'COST_VOICE_FIX_PMIN2',
'COST_VOICE_FIX_PMIN3',
'COST_VOICE_FIX_PMIN4',
'COST_VOICE_INT_PMIN2',
'COST_VOICE_INT_PMIN3',
'COST_VOICE_INT_PMIN4',
'COST_VOICE_ROAMING_PMIN2',
'COST_VOICE_ROAMING_PMIN3',
'COST_VOICE_ROAMING_PMIN4',
'COST_VOICE_TOTAL_PMIN2',
'COST_VOICE_TOTAL_PMIN3',
'COST_VOICE_TOTAL_PMIN4',
'COST_VOICE_VAS_PMIN2',
'COST_VOICE_VAS_PMIN3',
'COST_VOICE_VAS_PMIN4',
'COST_SMS_ONNET_PMIN2',
'COST_SMS_ONNET_PMIN3',
'COST_SMS_ONNET_PMIN4',
'COST_SMS_OFFNET_PMIN2',
'COST_SMS_OFFNET_PMIN3',
'COST_SMS_OFFNET_PMIN4',
'COST_SMS_VIP_PMIN2',
'COST_SMS_VIP_PMIN3',
'COST_SMS_VIP_PMIN4',
'COST_SMS_TELE2_PMIN2',
'COST_SMS_TELE2_PMIN3',
'COST_SMS_TELE2_PMIN4',
'COST_SMS_TOMATO_PMIN2',
'COST_SMS_TOMATO_PMIN3',
'COST_SMS_TOMATO_PMIN4',
'COST_SMS_BONBON_PMIN2',
'COST_SMS_BONBON_PMIN3',
'COST_SMS_BONBON_PMIN4',
'COST_SMS_TOTAL_PMIN2',
'COST_SMS_TOTAL_PMIN3',
'COST_SMS_TOTAL_PMIN4',
'COST_SMS_ROAMING_PMIN2',
'COST_SMS_ROAMING_PMIN3',
'COST_SMS_ROAMING_PMIN4',
'COST_MMS_TOTAL_PMIN2',
'COST_MMS_TOTAL_PMIN3',
'COST_MMS_TOTAL_PMIN4',
'COST_DATA_PMIN2',
'COST_DATA_PMIN3',
'COST_DATA_PMIN4',
'COST_DATA_NATIONAL_PMIN2',
'COST_DATA_NATIONAL_PMIN3',
'COST_DATA_NATIONAL_PMIN4'

]

In [ ]:
mob_09_tele_avg = mob_09_tele.select("oib",*L3)
        

In [ ]:
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ALL_CALLS_PMIN2",F.col("VOICE_ALL_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ALL_CALLS_PMIN3",F.col("VOICE_ALL_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ALL_CALLS_PMIN4",F.col("VOICE_ALL_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TMHR_CALLS_PMIN2",F.col("VOICE_TMHR_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TMHR_CALLS_PMIN3",F.col("VOICE_TMHR_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TMHR_CALLS_PMIN4",F.col("VOICE_TMHR_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_BONBON_CALLS_PMIN2",F.col("VOICE_BONBON_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_BONBON_CALLS_PMIN3",F.col("VOICE_BONBON_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_BONBON_CALLS_PMIN4",F.col("VOICE_BONBON_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VIP_CALLS_PMIN2",F.col("VOICE_VIP_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VIP_CALLS_PMIN3",F.col("VOICE_VIP_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VIP_CALLS_PMIN4",F.col("VOICE_VIP_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TELE2_CALLS_PMIN2",F.col("VOICE_TELE2_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TELE2_CALLS_PMIN3",F.col("VOICE_TELE2_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TELE2_CALLS_PMIN4",F.col("VOICE_TELE2_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TOMATO_CALLS_PMIN2",F.col("VOICE_TOMATO_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TOMATO_CALLS_PMIN3",F.col("VOICE_TOMATO_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TOMATO_CALLS_PMIN4",F.col("VOICE_TOMATO_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_INTRNTIONAL_CALLS_PMIN2",F.col("VOICE_INTRNTIONAL_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_INTRNTIONAL_CALLS_PMIN3",F.col("VOICE_INTRNTIONAL_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_INTRNTIONAL_CALLS_PMIN4",F.col("VOICE_INTRNTIONAL_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_FIXED_CALLS_MIN2",F.col("VOICE_FIXED_CALLS_MIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_FIXED_CALLS_MIN3",F.col("VOICE_FIXED_CALLS_MIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_FIXED_CALLS_MIN4",F.col("VOICE_FIXED_CALLS_MIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VAS_CALLS_PMIN2",F.col("VOICE_VAS_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VAS_CALLS_PMIN3",F.col("VOICE_VAS_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VAS_CALLS_PMIN4",F.col("VOICE_VAS_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ROAMING_CALLS_PMIN2",F.col("VOICE_ROAMING_CALLS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ROAMING_CALLS_PMIN3",F.col("VOICE_ROAMING_CALLS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ROAMING_CALLS_PMIN4",F.col("VOICE_ROAMING_CALLS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ALL_MIN_PMIN2",F.col("VOICE_ALL_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ALL_MIN_PMIN3",F.col("VOICE_ALL_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ALL_MIN_PMIN4",F.col("VOICE_ALL_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TMHR_MIN_PMIN2",F.col("VOICE_TMHR_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TMHR_MIN_PMIN3",F.col("VOICE_TMHR_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TMHR_MIN_PMIN4",F.col("VOICE_TMHR_MIN_PMIN4").cast(IntegerType()))
#mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_BONBON_MIN_PMIN_2",F.col("VOICE_BONBON_MIN_PMIN_2").cast(IntegerType()))
#mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_BONBON_MIN_PMIN_3",F.col("VOICE_BONBON_MIN_PMIN_3").cast(IntegerType()))
#mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_BONBON_MIN_PMIN_4",F.col("VOICE_BONBON_MIN_PMIN_4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VIP_MIN_PMIN2",F.col("VOICE_VIP_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VIP_MIN_PMIN3",F.col("VOICE_VIP_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VIP_MIN_PMIN4",F.col("VOICE_VIP_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TELE2_MIN_PMIN2",F.col("VOICE_TELE2_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TELE2_MIN_PMIN3",F.col("VOICE_TELE2_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TELE2_MIN_PMIN4",F.col("VOICE_TELE2_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TOMATO_MIN_PMIN2",F.col("VOICE_TOMATO_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TOMATO_MIN_PMIN3",F.col("VOICE_TOMATO_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_TOMATO_MIN_PMIN4",F.col("VOICE_TOMATO_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_INTERNATIONAL_MIN_PMIN2",F.col("VOICE_INTERNATIONAL_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_INTERNATIONAL_MIN_PMIN3",F.col("VOICE_INTERNATIONAL_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_INTERNATIONAL_MIN_PMIN4",F.col("VOICE_INTERNATIONAL_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_FIXED_MIN_PMIN2",F.col("VOICE_FIXED_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_FIXED_MIN_PMIN3",F.col("VOICE_FIXED_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_FIXED_MIN_PMIN4",F.col("VOICE_FIXED_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VAS_MIN_PMIN2",F.col("VOICE_VAS_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VAS_MIN_PMIN3",F.col("VOICE_VAS_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_VAS_MIN_PMIN4",F.col("VOICE_VAS_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ROAMING_MIN_PMIN2",F.col("VOICE_ROAMING_MIN_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ROAMING_MIN_PMIN3",F.col("VOICE_ROAMING_MIN_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("VOICE_ROAMING_MIN_PMIN4",F.col("VOICE_ROAMING_MIN_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_ALL_PMIN2",F.col("SMS_ALL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_ALL_PMIN3",F.col("SMS_ALL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_ALL_PMIN4",F.col("SMS_ALL_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TMHR_PMIN2",F.col("SMS_TMHR_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TMHR_PMIN3",F.col("SMS_TMHR_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TMHR_PMIN4",F.col("SMS_TMHR_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_BONBON_PMIN2",F.col("SMS_BONBON_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_BONBON_PMIN3",F.col("SMS_BONBON_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_BONBON_PMIN4",F.col("SMS_BONBON_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_VIP_PMIN2",F.col("SMS_VIP_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_VIP_PMIN3",F.col("SMS_VIP_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_VIP_PMIN4",F.col("SMS_VIP_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TELE2_PMIN2",F.col("SMS_TELE2_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TELE2_PMIN3",F.col("SMS_TELE2_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TELE2_PMIN4",F.col("SMS_TELE2_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TOMATO_PMIN2",F.col("SMS_TOMATO_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TOMATO_PMIN3",F.col("SMS_TOMATO_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_TOMATO_PMIN4",F.col("SMS_TOMATO_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_INTERNATIONAL_PMIN2",F.col("SMS_INTERNATIONAL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_INTERNATIONAL_PMIN3",F.col("SMS_INTERNATIONAL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_INTERNATIONAL_PMIN4",F.col("SMS_INTERNATIONAL_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_VAS_PMIN2",F.col("SMS_VAS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_VAS_PMIN3",F.col("SMS_VAS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_VAS_PMIN4",F.col("SMS_VAS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_ROAMING_PMIN2",F.col("SMS_ROAMING_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_ROAMING_PMIN3",F.col("SMS_ROAMING_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("SMS_ROAMING_PMIN4",F.col("SMS_ROAMING_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_NATIONAL_VOLUME_MB_PMIN2",F.col("DATA_NATIONAL_VOLUME_MB_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_NATIONAL_VOLUME_MB_PMIN3",F.col("DATA_NATIONAL_VOLUME_MB_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_NATIONAL_VOLUME_MB_PMIN4",F.col("DATA_NATIONAL_VOLUME_MB_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_NATIONAL_SESSIONS_PMIN2",F.col("DATA_NATIONAL_SESSIONS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_NATIONAL_SESSIONS_PMIN3",F.col("DATA_NATIONAL_SESSIONS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_NATIONAL_SESSIONS_PMIN4",F.col("DATA_NATIONAL_SESSIONS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_ROAMING_VOLUME_MB_PMIN2",F.col("DATA_ROAMING_VOLUME_MB_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_ROAMING_VOLUME_MB_PMIN3",F.col("DATA_ROAMING_VOLUME_MB_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_ROAMING_VOLUME_MB_PMIN4",F.col("DATA_ROAMING_VOLUME_MB_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_ROAMING_SESSIONS_PMIN2",F.col("DATA_ROAMING_SESSIONS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_ROAMING_SESSIONS_PMIN3",F.col("DATA_ROAMING_SESSIONS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("DATA_ROAMING_SESSIONS_PMIN4",F.col("DATA_ROAMING_SESSIONS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_ADMIN_ACT_PMIN2",F.col("INT_CALL_IN_ADMIN_ACT_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_ADMIN_ACT_PMIN3",F.col("INT_CALL_IN_ADMIN_ACT_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_ADMIN_ACT_PMIN4",F.col("INT_CALL_IN_ADMIN_ACT_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_HRC_PMIN2",F.col("INT_CALL_IN_HRC_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_HRC_PMIN3",F.col("INT_CALL_IN_HRC_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_HRC_PMIN4",F.col("INT_CALL_IN_HRC_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_ICCA_TIM_PMIN2",F.col("INT_CALL_IN_ICCA_TIM_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_ICCA_TIM_PMIN3",F.col("INT_CALL_IN_ICCA_TIM_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_ICCA_TIM_PMIN4",F.col("INT_CALL_IN_ICCA_TIM_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_INFO_PMIN2",F.col("INT_CALL_IN_INFO_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_INFO_PMIN3",F.col("INT_CALL_IN_INFO_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_INFO_PMIN4",F.col("INT_CALL_IN_INFO_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_INTERACTION_PMIN2",F.col("INT_CALL_IN_INTERACTION_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_INTERACTION_PMIN3",F.col("INT_CALL_IN_INTERACTION_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_INTERACTION_PMIN4",F.col("INT_CALL_IN_INTERACTION_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_USLUGE_KAMP_PMIN2",F.col("INT_CALL_IN_USLUGE_KAMP_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_USLUGE_KAMP_PMIN3",F.col("INT_CALL_IN_USLUGE_KAMP_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_USLUGE_KAMP_PMIN4",F.col("INT_CALL_IN_USLUGE_KAMP_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_OSTALO_PMIN2",F.col("INT_CALL_IN_OSTALO_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_OSTALO_PMIN3",F.col("INT_CALL_IN_OSTALO_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_OSTALO_PMIN4",F.col("INT_CALL_IN_OSTALO_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_RACUNI_PMIN2",F.col("INT_CALL_IN_RACUNI_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_RACUNI_PMIN3",F.col("INT_CALL_IN_RACUNI_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_RACUNI_PMIN4",F.col("INT_CALL_IN_RACUNI_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_POS_KOR_PMIN2",F.col("INT_CALL_IN_POS_KOR_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_POS_KOR_PMIN3",F.col("INT_CALL_IN_POS_KOR_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_POS_KOR_PMIN4",F.col("INT_CALL_IN_POS_KOR_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_PRIGOVOR_PMIN2",F.col("INT_CALL_IN_PRIGOVOR_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_PRIGOVOR_PMIN3",F.col("INT_CALL_IN_PRIGOVOR_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_PRIGOVOR_PMIN4",F.col("INT_CALL_IN_PRIGOVOR_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_UGOVOR_PMIN2",F.col("INT_CALL_IN_UGOVOR_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_UGOVOR_PMIN3",F.col("INT_CALL_IN_UGOVOR_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_UGOVOR_PMIN4",F.col("INT_CALL_IN_UGOVOR_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_SAVEDESK_PMIN2",F.col("INT_CALL_IN_SAVEDESK_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_SAVEDESK_PMIN3",F.col("INT_CALL_IN_SAVEDESK_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_SAVEDESK_PMIN4",F.col("INT_CALL_IN_SAVEDESK_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TEHNICKA_PMIN2",F.col("INT_CALL_IN_TEHNICKA_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TEHNICKA_PMIN3",F.col("INT_CALL_IN_TEHNICKA_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TEHNICKA_PMIN4",F.col("INT_CALL_IN_TEHNICKA_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TELESALES_PMIN2",F.col("INT_CALL_IN_TELESALES_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TELESALES_PMIN3",F.col("INT_CALL_IN_TELESALES_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TELESALES_PMIN4",F.col("INT_CALL_IN_TELESALES_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TRIM_TIM_PMIN2",F.col("INT_CALL_IN_TRIM_TIM_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TRIM_TIM_PMIN3",F.col("INT_CALL_IN_TRIM_TIM_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_TRIM_TIM_PMIN4",F.col("INT_CALL_IN_TRIM_TIM_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_X7_PMIN2",F.col("INT_CALL_IN_X7_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_X7_PMIN3",F.col("INT_CALL_IN_X7_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INT_CALL_IN_X7_PMIN4",F.col("INT_CALL_IN_X7_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("QTY_DAYS_INTERACTING_PMIN2",F.col("QTY_DAYS_INTERACTING_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("QTY_DAYS_INTERACTING_PMIN3",F.col("QTY_DAYS_INTERACTING_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("QTY_DAYS_INTERACTING_PMIN4",F.col("QTY_DAYS_INTERACTING_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INVOICED_PMIN2",F.col("INVOICED_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INVOICED_PMIN3",F.col("INVOICED_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("INVOICED_PMIN4",F.col("INVOICED_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_PMIN2",F.col("REVENUE_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_PMIN3",F.col("REVENUE_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_PMIN4",F.col("REVENUE_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("ROAMING_AMOUNT_PMIN2",F.col("ROAMING_AMOUNT_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("ROAMING_AMOUNT_PMIN3",F.col("ROAMING_AMOUNT_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("ROAMING_AMOUNT_PMIN4",F.col("ROAMING_AMOUNT_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SUBSCRIPTION_PMIN2",F.col("REVENUE_SUBSCRIPTION_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SUBSCRIPTION_PMIN3",F.col("REVENUE_SUBSCRIPTION_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SUBSCRIPTION_PMIN4",F.col("REVENUE_SUBSCRIPTION_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_VOICE_PMIN2",F.col("REVENUE_VOICE_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_VOICE_PMIN3",F.col("REVENUE_VOICE_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_VOICE_PMIN4",F.col("REVENUE_VOICE_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SMS_PMIN2",F.col("REVENUE_SMS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SMS_PMIN3",F.col("REVENUE_SMS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SMS_PMIN4",F.col("REVENUE_SMS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_DATA_NATIONAL_PMIN2",F.col("REVENUE_DATA_NATIONAL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_DATA_NATIONAL_PMIN3",F.col("REVENUE_DATA_NATIONAL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_DATA_NATIONAL_PMIN4",F.col("REVENUE_DATA_NATIONAL_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_DATA_ROAMING_PMIN2",F.col("REVENUE_DATA_ROAMING_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_DATA_ROAMING_PMIN3",F.col("REVENUE_DATA_ROAMING_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_DATA_ROAMING_PMIN4",F.col("REVENUE_DATA_ROAMING_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_VOICE_VAS_PMIN2",F.col("REVENUE_VOICE_VAS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_VOICE_VAS_PMIN3",F.col("REVENUE_VOICE_VAS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_VOICE_VAS_PMIN4",F.col("REVENUE_VOICE_VAS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SMS_VAS_PMIN2",F.col("REVENUE_SMS_VAS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SMS_VAS_PMIN3",F.col("REVENUE_SMS_VAS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_SMS_VAS_PMIN4",F.col("REVENUE_SMS_VAS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_GSM_LATE_FEE_PMIN2",F.col("REVENUE_GSM_LATE_FEE_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_GSM_LATE_FEE_PMIN3",F.col("REVENUE_GSM_LATE_FEE_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_GSM_LATE_FEE_PMIN4",F.col("REVENUE_GSM_LATE_FEE_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_GSM_OTHER_PMIN2",F.col("REVENUE_GSM_OTHER_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_GSM_OTHER_PMIN3",F.col("REVENUE_GSM_OTHER_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("REVENUE_GSM_OTHER_PMIN4",F.col("REVENUE_GSM_OTHER_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_ONNET_PMIN2",F.col("COST_VOICE_ONNET_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_ONNET_PMIN3",F.col("COST_VOICE_ONNET_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_ONNET_PMIN4",F.col("COST_VOICE_ONNET_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_OFFNET_PMIN2",F.col("COST_VOICE_OFFNET_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_OFFNET_PMIN3",F.col("COST_VOICE_OFFNET_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_OFFNET_PMIN4",F.col("COST_VOICE_OFFNET_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_VIP_PMIN2",F.col("COST_VOICE_VIP_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_VIP_PMIN3",F.col("COST_VOICE_VIP_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_VIP_PMIN4",F.col("COST_VOICE_VIP_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TELE2_PMIN2",F.col("COST_VOICE_TELE2_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TELE2_PMIN3",F.col("COST_VOICE_TELE2_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TELE2_PMIN4",F.col("COST_VOICE_TELE2_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TOMATO_PMIN2",F.col("COST_VOICE_TOMATO_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TOMATO_PMIN3",F.col("COST_VOICE_TOMATO_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TOMATO_PMIN4",F.col("COST_VOICE_TOMATO_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_BONBON_PMIN2",F.col("COST_VOICE_BONBON_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_BONBON_PMIN3",F.col("COST_VOICE_BONBON_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_BONBON_PMIN4",F.col("COST_VOICE_BONBON_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_FIX_PMIN2",F.col("COST_VOICE_FIX_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_FIX_PMIN3",F.col("COST_VOICE_FIX_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_FIX_PMIN4",F.col("COST_VOICE_FIX_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_INT_PMIN2",F.col("COST_VOICE_INT_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_INT_PMIN3",F.col("COST_VOICE_INT_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_INT_PMIN4",F.col("COST_VOICE_INT_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_ROAMING_PMIN2",F.col("COST_VOICE_ROAMING_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_ROAMING_PMIN3",F.col("COST_VOICE_ROAMING_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_ROAMING_PMIN4",F.col("COST_VOICE_ROAMING_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TOTAL_PMIN2",F.col("COST_VOICE_TOTAL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TOTAL_PMIN3",F.col("COST_VOICE_TOTAL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_TOTAL_PMIN4",F.col("COST_VOICE_TOTAL_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_VAS_PMIN2",F.col("COST_VOICE_VAS_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_VAS_PMIN3",F.col("COST_VOICE_VAS_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_VOICE_VAS_PMIN4",F.col("COST_VOICE_VAS_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_ONNET_PMIN2",F.col("COST_SMS_ONNET_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_ONNET_PMIN3",F.col("COST_SMS_ONNET_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_ONNET_PMIN4",F.col("COST_SMS_ONNET_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_OFFNET_PMIN2",F.col("COST_SMS_OFFNET_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_OFFNET_PMIN3",F.col("COST_SMS_OFFNET_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_OFFNET_PMIN4",F.col("COST_SMS_OFFNET_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_VIP_PMIN2",F.col("COST_SMS_VIP_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_VIP_PMIN3",F.col("COST_SMS_VIP_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_VIP_PMIN4",F.col("COST_SMS_VIP_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TELE2_PMIN2",F.col("COST_SMS_TELE2_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TELE2_PMIN3",F.col("COST_SMS_TELE2_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TELE2_PMIN4",F.col("COST_SMS_TELE2_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TOMATO_PMIN2",F.col("COST_SMS_TOMATO_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TOMATO_PMIN3",F.col("COST_SMS_TOMATO_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TOMATO_PMIN4",F.col("COST_SMS_TOMATO_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_BONBON_PMIN2",F.col("COST_SMS_BONBON_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_BONBON_PMIN3",F.col("COST_SMS_BONBON_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_BONBON_PMIN4",F.col("COST_SMS_BONBON_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TOTAL_PMIN2",F.col("COST_SMS_TOTAL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TOTAL_PMIN3",F.col("COST_SMS_TOTAL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_TOTAL_PMIN4",F.col("COST_SMS_TOTAL_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_ROAMING_PMIN2",F.col("COST_SMS_ROAMING_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_ROAMING_PMIN3",F.col("COST_SMS_ROAMING_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_SMS_ROAMING_PMIN4",F.col("COST_SMS_ROAMING_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_MMS_TOTAL_PMIN2",F.col("COST_MMS_TOTAL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_MMS_TOTAL_PMIN3",F.col("COST_MMS_TOTAL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_MMS_TOTAL_PMIN4",F.col("COST_MMS_TOTAL_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_DATA_PMIN2",F.col("COST_DATA_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_DATA_PMIN3",F.col("COST_DATA_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_DATA_PMIN4",F.col("COST_DATA_PMIN4").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_DATA_NATIONAL_PMIN2",F.col("COST_DATA_NATIONAL_PMIN2").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_DATA_NATIONAL_PMIN3",F.col("COST_DATA_NATIONAL_PMIN3").cast(IntegerType()))
mob_09_tele_avg = mob_09_tele_avg.withColumn("COST_DATA_NATIONAL_PMIN4",F.col("COST_DATA_NATIONAL_PMIN4").cast(IntegerType()))

In [ ]:
mob_09_tele_avg11 = mob_09_tele_avg.groupBy("oib").agg(

F.avg("VOICE_ALL_CALLS_PMIN2").alias("VOICE_ALL_CALLS_PMIN2"),
F.avg("VOICE_ALL_CALLS_PMIN3").alias("VOICE_ALL_CALLS_PMIN3"),
F.avg("VOICE_ALL_CALLS_PMIN4").alias("VOICE_ALL_CALLS_PMIN4"),
F.avg("VOICE_TMHR_CALLS_PMIN2").alias("VOICE_TMHR_CALLS_PMIN2"),
F.avg("VOICE_TMHR_CALLS_PMIN3").alias("VOICE_TMHR_CALLS_PMIN3"),
F.avg("VOICE_TMHR_CALLS_PMIN4").alias("VOICE_TMHR_CALLS_PMIN4"),
F.avg("VOICE_BONBON_CALLS_PMIN2").alias("VOICE_BONBON_CALLS_PMIN2"),
F.avg("VOICE_BONBON_CALLS_PMIN3").alias("VOICE_BONBON_CALLS_PMIN3"),
F.avg("VOICE_BONBON_CALLS_PMIN4").alias("VOICE_BONBON_CALLS_PMIN4"),
F.avg("VOICE_VIP_CALLS_PMIN2").alias("VOICE_VIP_CALLS_PMIN2"),
F.avg("VOICE_VIP_CALLS_PMIN3").alias("VOICE_VIP_CALLS_PMIN3"),
F.avg("VOICE_VIP_CALLS_PMIN4").alias("VOICE_VIP_CALLS_PMIN4"),
# F.avg("VOICE_TELE2_CALLS_PMIN2").alias("VOICE_TELE2_CALLS_PMIN2"),
# F.avg("VOICE_TELE2_CALLS_PMIN3").alias("VOICE_TELE2_CALLS_PMIN3"),
# F.avg("VOICE_TELE2_CALLS_PMIN4").alias("VOICE_TELE2_CALLS_PMIN4"),
F.avg("VOICE_TOMATO_CALLS_PMIN2").alias("VOICE_TOMATO_CALLS_PMIN2"),
F.avg("VOICE_TOMATO_CALLS_PMIN3").alias("VOICE_TOMATO_CALLS_PMIN3"),
F.avg("VOICE_TOMATO_CALLS_PMIN4").alias("VOICE_TOMATO_CALLS_PMIN4"),
F.avg("VOICE_INTRNTIONAL_CALLS_PMIN2").alias("VOICE_INTRNTIONAL_CALLS_PMIN2"),
F.avg("VOICE_INTRNTIONAL_CALLS_PMIN3").alias("VOICE_INTRNTIONAL_CALLS_PMIN3"),
F.avg("VOICE_INTRNTIONAL_CALLS_PMIN4").alias("VOICE_INTRNTIONAL_CALLS_PMIN4"),
F.avg("VOICE_FIXED_CALLS_MIN2").alias("VOICE_FIXED_CALLS_MIN2"),
F.avg("VOICE_FIXED_CALLS_MIN3").alias("VOICE_FIXED_CALLS_MIN3"),
F.avg("VOICE_FIXED_CALLS_MIN4").alias("VOICE_FIXED_CALLS_MIN4"),
F.avg("VOICE_VAS_CALLS_PMIN2").alias("VOICE_VAS_CALLS_PMIN2"),
F.avg("VOICE_VAS_CALLS_PMIN3").alias("VOICE_VAS_CALLS_PMIN3"),
F.avg("VOICE_VAS_CALLS_PMIN4").alias("VOICE_VAS_CALLS_PMIN4"),
F.avg("VOICE_ROAMING_CALLS_PMIN2").alias("VOICE_ROAMING_CALLS_PMIN2"),
F.avg("VOICE_ROAMING_CALLS_PMIN3").alias("VOICE_ROAMING_CALLS_PMIN3"),
F.avg("VOICE_ROAMING_CALLS_PMIN4").alias("VOICE_ROAMING_CALLS_PMIN4"),
F.avg("VOICE_ALL_MIN_PMIN2").alias("VOICE_ALL_MIN_PMIN2"),
F.avg("VOICE_ALL_MIN_PMIN3").alias("VOICE_ALL_MIN_PMIN3"),
F.avg("VOICE_ALL_MIN_PMIN4").alias("VOICE_ALL_MIN_PMIN4"),
F.avg("VOICE_TMHR_MIN_PMIN2").alias("VOICE_TMHR_MIN_PMIN2"),
F.avg("VOICE_TMHR_MIN_PMIN3").alias("VOICE_TMHR_MIN_PMIN3"),
F.avg("VOICE_TMHR_MIN_PMIN4").alias("VOICE_TMHR_MIN_PMIN4"),
F.avg("VOICE_BONBON_MIN_PMIN_2").alias("VOICE_BONBON_MIN_PMIN_2"),
F.avg("VOICE_BONBON_MIN_PMIN_3").alias("VOICE_BONBON_MIN_PMIN_3"),
F.avg("VOICE_BONBON_MIN_PMIN_4").alias("VOICE_BONBON_MIN_PMIN_4"),
F.avg("VOICE_VIP_MIN_PMIN2").alias("VOICE_VIP_MIN_PMIN2"),
F.avg("VOICE_VIP_MIN_PMIN3").alias("VOICE_VIP_MIN_PMIN3"),
F.avg("VOICE_VIP_MIN_PMIN4").alias("VOICE_VIP_MIN_PMIN4"),
F.avg("VOICE_TELE2_MIN_PMIN2").alias("VOICE_TELE2_MIN_PMIN2"),
F.avg("VOICE_TELE2_MIN_PMIN3").alias("VOICE_TELE2_MIN_PMIN3"),
F.avg("VOICE_TELE2_MIN_PMIN4").alias("VOICE_TELE2_MIN_PMIN4"),
F.avg("VOICE_TOMATO_MIN_PMIN2").alias("VOICE_TOMATO_MIN_PMIN2"),
F.avg("VOICE_TOMATO_MIN_PMIN3").alias("VOICE_TOMATO_MIN_PMIN3"),
F.avg("VOICE_TOMATO_MIN_PMIN4").alias("VOICE_TOMATO_MIN_PMIN4"),
F.avg("VOICE_INTERNATIONAL_MIN_PMIN2").alias("VOICE_INTERNATIONAL_MIN_PMIN2"),
F.avg("VOICE_INTERNATIONAL_MIN_PMIN3").alias("VOICE_INTERNATIONAL_MIN_PMIN3"),
F.avg("VOICE_INTERNATIONAL_MIN_PMIN4").alias("VOICE_INTERNATIONAL_MIN_PMIN4"),
F.avg("VOICE_FIXED_MIN_PMIN2").alias("VOICE_FIXED_MIN_PMIN2"),
F.avg("VOICE_FIXED_MIN_PMIN3").alias("VOICE_FIXED_MIN_PMIN3"),
F.avg("VOICE_FIXED_MIN_PMIN4").alias("VOICE_FIXED_MIN_PMIN4"),
F.avg("VOICE_VAS_MIN_PMIN2").alias("VOICE_VAS_MIN_PMIN2"),
F.avg("VOICE_VAS_MIN_PMIN3").alias("VOICE_VAS_MIN_PMIN3"),
F.avg("VOICE_VAS_MIN_PMIN4").alias("VOICE_VAS_MIN_PMIN4"),
F.avg("VOICE_ROAMING_MIN_PMIN2").alias("VOICE_ROAMING_MIN_PMIN2"),
F.avg("VOICE_ROAMING_MIN_PMIN3").alias("VOICE_ROAMING_MIN_PMIN3"),
F.avg("VOICE_ROAMING_MIN_PMIN4").alias("VOICE_ROAMING_MIN_PMIN4"),
F.avg("SMS_ALL_PMIN2").alias("SMS_ALL_PMIN2"),
F.avg("SMS_ALL_PMIN3").alias("SMS_ALL_PMIN3"),
F.avg("SMS_ALL_PMIN4").alias("SMS_ALL_PMIN4"),
F.avg("SMS_TMHR_PMIN2").alias("SMS_TMHR_PMIN2"),
F.avg("SMS_TMHR_PMIN3").alias("SMS_TMHR_PMIN3"),
F.avg("SMS_TMHR_PMIN4").alias("SMS_TMHR_PMIN4"),
F.avg("SMS_BONBON_PMIN2").alias("SMS_BONBON_PMIN2"),
F.avg("SMS_BONBON_PMIN3").alias("SMS_BONBON_PMIN3"),
F.avg("SMS_BONBON_PMIN4").alias("SMS_BONBON_PMIN4"),
F.avg("SMS_VIP_PMIN2").alias("SMS_VIP_PMIN2"),
F.avg("SMS_VIP_PMIN3").alias("SMS_VIP_PMIN3"),
F.avg("SMS_VIP_PMIN4").alias("SMS_VIP_PMIN4"),
F.avg("SMS_TELE2_PMIN2").alias("SMS_TELE2_PMIN2"),
F.avg("SMS_TELE2_PMIN3").alias("SMS_TELE2_PMIN3"),
F.avg("SMS_TELE2_PMIN4").alias("SMS_TELE2_PMIN4"),
F.avg("SMS_TOMATO_PMIN2").alias("SMS_TOMATO_PMIN2"),
F.avg("SMS_TOMATO_PMIN3").alias("SMS_TOMATO_PMIN3"),
F.avg("SMS_TOMATO_PMIN4").alias("SMS_TOMATO_PMIN4"),
F.avg("SMS_INTERNATIONAL_PMIN2").alias("SMS_INTERNATIONAL_PMIN2"),
F.avg("SMS_INTERNATIONAL_PMIN3").alias("SMS_INTERNATIONAL_PMIN3"),
F.avg("SMS_INTERNATIONAL_PMIN4").alias("SMS_INTERNATIONAL_PMIN4"),
F.avg("SMS_VAS_PMIN2").alias("SMS_VAS_PMIN2"),
F.avg("SMS_VAS_PMIN3").alias("SMS_VAS_PMIN3"),
F.avg("SMS_VAS_PMIN4").alias("SMS_VAS_PMIN4"),
F.avg("SMS_ROAMING_PMIN2").alias("SMS_ROAMING_PMIN2"),
F.avg("SMS_ROAMING_PMIN3").alias("SMS_ROAMING_PMIN3"),
F.avg("SMS_ROAMING_PMIN4").alias("SMS_ROAMING_PMIN4"),
F.avg("DATA_NATIONAL_VOLUME_MB_PMIN2").alias("DATA_NATIONAL_VOLUME_MB_PMIN2"),
F.avg("DATA_NATIONAL_VOLUME_MB_PMIN3").alias("DATA_NATIONAL_VOLUME_MB_PMIN3"),
F.avg("DATA_NATIONAL_VOLUME_MB_PMIN4").alias("DATA_NATIONAL_VOLUME_MB_PMIN4"),
F.avg("DATA_NATIONAL_SESSIONS_PMIN2").alias("DATA_NATIONAL_SESSIONS_PMIN2"),
F.avg("DATA_NATIONAL_SESSIONS_PMIN3").alias("DATA_NATIONAL_SESSIONS_PMIN3"),
F.avg("DATA_NATIONAL_SESSIONS_PMIN4").alias("DATA_NATIONAL_SESSIONS_PMIN4"),
F.avg("DATA_ROAMING_VOLUME_MB_PMIN2").alias("DATA_ROAMING_VOLUME_MB_PMIN2"),
F.avg("DATA_ROAMING_VOLUME_MB_PMIN3").alias("DATA_ROAMING_VOLUME_MB_PMIN3"),
F.avg("DATA_ROAMING_VOLUME_MB_PMIN4").alias("DATA_ROAMING_VOLUME_MB_PMIN4"),
F.avg("DATA_ROAMING_SESSIONS_PMIN2").alias("DATA_ROAMING_SESSIONS_PMIN2"),
F.avg("DATA_ROAMING_SESSIONS_PMIN3").alias("DATA_ROAMING_SESSIONS_PMIN3"),
F.avg("DATA_ROAMING_SESSIONS_PMIN4").alias("DATA_ROAMING_SESSIONS_PMIN4"),
F.avg("INT_CALL_IN_ADMIN_ACT_PMIN2").alias("INT_CALL_IN_ADMIN_ACT_PMIN2"),
F.avg("INT_CALL_IN_ADMIN_ACT_PMIN3").alias("INT_CALL_IN_ADMIN_ACT_PMIN3"),
F.avg("INT_CALL_IN_ADMIN_ACT_PMIN4").alias("INT_CALL_IN_ADMIN_ACT_PMIN4"),
F.avg("INT_CALL_IN_HRC_PMIN2").alias("INT_CALL_IN_HRC_PMIN2"),
F.avg("INT_CALL_IN_HRC_PMIN3").alias("INT_CALL_IN_HRC_PMIN3"),
F.avg("INT_CALL_IN_HRC_PMIN4").alias("INT_CALL_IN_HRC_PMIN4"),
F.avg("INT_CALL_IN_ICCA_TIM_PMIN2").alias("INT_CALL_IN_ICCA_TIM_PMIN2"),
F.avg("INT_CALL_IN_ICCA_TIM_PMIN3").alias("INT_CALL_IN_ICCA_TIM_PMIN3"),
F.avg("INT_CALL_IN_ICCA_TIM_PMIN4").alias("INT_CALL_IN_ICCA_TIM_PMIN4"),
F.avg("INT_CALL_IN_INFO_PMIN2").alias("INT_CALL_IN_INFO_PMIN2"),
F.avg("INT_CALL_IN_INFO_PMIN3").alias("INT_CALL_IN_INFO_PMIN3"),
F.avg("INT_CALL_IN_INFO_PMIN4").alias("INT_CALL_IN_INFO_PMIN4"),
F.avg("INT_CALL_IN_INTERACTION_PMIN2").alias("INT_CALL_IN_INTERACTION_PMIN2"),
F.avg("INT_CALL_IN_INTERACTION_PMIN3").alias("INT_CALL_IN_INTERACTION_PMIN3"),
F.avg("INT_CALL_IN_INTERACTION_PMIN4").alias("INT_CALL_IN_INTERACTION_PMIN4"),
F.avg("INT_CALL_IN_USLUGE_KAMP_PMIN2").alias("INT_CALL_IN_USLUGE_KAMP_PMIN2"),
F.avg("INT_CALL_IN_USLUGE_KAMP_PMIN3").alias("INT_CALL_IN_USLUGE_KAMP_PMIN3"),
F.avg("INT_CALL_IN_USLUGE_KAMP_PMIN4").alias("INT_CALL_IN_USLUGE_KAMP_PMIN4"),
F.avg("INT_CALL_IN_OSTALO_PMIN2").alias("INT_CALL_IN_OSTALO_PMIN2"),
F.avg("INT_CALL_IN_OSTALO_PMIN3").alias("INT_CALL_IN_OSTALO_PMIN3"),
F.avg("INT_CALL_IN_OSTALO_PMIN4").alias("INT_CALL_IN_OSTALO_PMIN4"),
F.avg("INT_CALL_IN_RACUNI_PMIN2").alias("INT_CALL_IN_RACUNI_PMIN2"),
F.avg("INT_CALL_IN_RACUNI_PMIN3").alias("INT_CALL_IN_RACUNI_PMIN3"),
F.avg("INT_CALL_IN_RACUNI_PMIN4").alias("INT_CALL_IN_RACUNI_PMIN4"),
F.avg("INT_CALL_IN_POS_KOR_PMIN2").alias("INT_CALL_IN_POS_KOR_PMIN2"),
F.avg("INT_CALL_IN_POS_KOR_PMIN3").alias("INT_CALL_IN_POS_KOR_PMIN3"),
F.avg("INT_CALL_IN_POS_KOR_PMIN4").alias("INT_CALL_IN_POS_KOR_PMIN4"),
F.avg("INT_CALL_IN_PRIGOVOR_PMIN2").alias("INT_CALL_IN_PRIGOVOR_PMIN2"),
F.avg("INT_CALL_IN_PRIGOVOR_PMIN3").alias("INT_CALL_IN_PRIGOVOR_PMIN3"),
F.avg("INT_CALL_IN_PRIGOVOR_PMIN4").alias("INT_CALL_IN_PRIGOVOR_PMIN4"),
F.avg("INT_CALL_IN_UGOVOR_PMIN2").alias("INT_CALL_IN_UGOVOR_PMIN2"),
F.avg("INT_CALL_IN_UGOVOR_PMIN3").alias("INT_CALL_IN_UGOVOR_PMIN3"),
F.avg("INT_CALL_IN_UGOVOR_PMIN4").alias("INT_CALL_IN_UGOVOR_PMIN4"),
F.avg("INT_CALL_IN_SAVEDESK_PMIN2").alias("INT_CALL_IN_SAVEDESK_PMIN2"),
F.avg("INT_CALL_IN_SAVEDESK_PMIN3").alias("INT_CALL_IN_SAVEDESK_PMIN3"),
F.avg("INT_CALL_IN_SAVEDESK_PMIN4").alias("INT_CALL_IN_SAVEDESK_PMIN4"),
F.avg("INT_CALL_IN_TEHNICKA_PMIN2").alias("INT_CALL_IN_TEHNICKA_PMIN2"),
F.avg("INT_CALL_IN_TEHNICKA_PMIN3").alias("INT_CALL_IN_TEHNICKA_PMIN3"),
F.avg("INT_CALL_IN_TEHNICKA_PMIN4").alias("INT_CALL_IN_TEHNICKA_PMIN4"),
F.avg("INT_CALL_IN_TELESALES_PMIN2").alias("INT_CALL_IN_TELESALES_PMIN2"),
F.avg("INT_CALL_IN_TELESALES_PMIN3").alias("INT_CALL_IN_TELESALES_PMIN3"),
F.avg("INT_CALL_IN_TELESALES_PMIN4").alias("INT_CALL_IN_TELESALES_PMIN4"),
F.avg("INT_CALL_IN_TRIM_TIM_PMIN2").alias("INT_CALL_IN_TRIM_TIM_PMIN2"),
F.avg("INT_CALL_IN_TRIM_TIM_PMIN3").alias("INT_CALL_IN_TRIM_TIM_PMIN3"),
F.avg("INT_CALL_IN_TRIM_TIM_PMIN4").alias("INT_CALL_IN_TRIM_TIM_PMIN4"),
F.avg("INT_CALL_IN_X7_PMIN2").alias("INT_CALL_IN_X7_PMIN2"),
F.avg("INT_CALL_IN_X7_PMIN3").alias("INT_CALL_IN_X7_PMIN3"),
F.avg("INT_CALL_IN_X7_PMIN4").alias("INT_CALL_IN_X7_PMIN4"),
F.avg("QTY_DAYS_INTERACTING_PMIN2").alias("QTY_DAYS_INTERACTING_PMIN2"),
F.avg("QTY_DAYS_INTERACTING_PMIN3").alias("QTY_DAYS_INTERACTING_PMIN3"),
F.avg("QTY_DAYS_INTERACTING_PMIN4").alias("QTY_DAYS_INTERACTING_PMIN4"),
F.avg("INVOICED_PMIN2").alias("INVOICED_PMIN2"),
F.avg("INVOICED_PMIN3").alias("INVOICED_PMIN3"),
F.avg("INVOICED_PMIN4").alias("INVOICED_PMIN4"),
F.avg("REVENUE_PMIN2").alias("REVENUE_PMIN2"),
F.avg("REVENUE_PMIN3").alias("REVENUE_PMIN3"),
F.avg("REVENUE_PMIN4").alias("REVENUE_PMIN4"),
F.avg("ROAMING_AMOUNT_PMIN2").alias("ROAMING_AMOUNT_PMIN2"),
F.avg("ROAMING_AMOUNT_PMIN3").alias("ROAMING_AMOUNT_PMIN3"),
F.avg("ROAMING_AMOUNT_PMIN4").alias("ROAMING_AMOUNT_PMIN4"),
F.avg("REVENUE_SUBSCRIPTION_PMIN2").alias("REVENUE_SUBSCRIPTION_PMIN2"),
F.avg("REVENUE_SUBSCRIPTION_PMIN3").alias("REVENUE_SUBSCRIPTION_PMIN3"),
F.avg("REVENUE_SUBSCRIPTION_PMIN4").alias("REVENUE_SUBSCRIPTION_PMIN4"),
F.avg("REVENUE_VOICE_PMIN2").alias("REVENUE_VOICE_PMIN2"),
F.avg("REVENUE_VOICE_PMIN3").alias("REVENUE_VOICE_PMIN3"),
F.avg("REVENUE_VOICE_PMIN4").alias("REVENUE_VOICE_PMIN4"),
F.avg("REVENUE_SMS_PMIN2").alias("REVENUE_SMS_PMIN2"),
F.avg("REVENUE_SMS_PMIN3").alias("REVENUE_SMS_PMIN3"),
F.avg("REVENUE_SMS_PMIN4").alias("REVENUE_SMS_PMIN4"),
F.avg("REVENUE_DATA_NATIONAL_PMIN2").alias("REVENUE_DATA_NATIONAL_PMIN2"),
F.avg("REVENUE_DATA_NATIONAL_PMIN3").alias("REVENUE_DATA_NATIONAL_PMIN3"),
F.avg("REVENUE_DATA_NATIONAL_PMIN4").alias("REVENUE_DATA_NATIONAL_PMIN4"),
F.avg("REVENUE_DATA_ROAMING_PMIN2").alias("REVENUE_DATA_ROAMING_PMIN2"),
F.avg("REVENUE_DATA_ROAMING_PMIN3").alias("REVENUE_DATA_ROAMING_PMIN3"),
F.avg("REVENUE_DATA_ROAMING_PMIN4").alias("REVENUE_DATA_ROAMING_PMIN4"),
F.avg("REVENUE_VOICE_VAS_PMIN2").alias("REVENUE_VOICE_VAS_PMIN2"),
F.avg("REVENUE_VOICE_VAS_PMIN3").alias("REVENUE_VOICE_VAS_PMIN3"),
F.avg("REVENUE_VOICE_VAS_PMIN4").alias("REVENUE_VOICE_VAS_PMIN4"),
F.avg("REVENUE_SMS_VAS_PMIN2").alias("REVENUE_SMS_VAS_PMIN2"),
F.avg("REVENUE_SMS_VAS_PMIN3").alias("REVENUE_SMS_VAS_PMIN3"),
F.avg("REVENUE_SMS_VAS_PMIN4").alias("REVENUE_SMS_VAS_PMIN4"),
F.avg("REVENUE_GSM_LATE_FEE_PMIN2").alias("REVENUE_GSM_LATE_FEE_PMIN2"),
F.avg("REVENUE_GSM_LATE_FEE_PMIN3").alias("REVENUE_GSM_LATE_FEE_PMIN3"),
F.avg("REVENUE_GSM_LATE_FEE_PMIN4").alias("REVENUE_GSM_LATE_FEE_PMIN4"),
F.avg("REVENUE_GSM_OTHER_PMIN2").alias("REVENUE_GSM_OTHER_PMIN2"),
F.avg("REVENUE_GSM_OTHER_PMIN3").alias("REVENUE_GSM_OTHER_PMIN3"),
F.avg("REVENUE_GSM_OTHER_PMIN4").alias("REVENUE_GSM_OTHER_PMIN4"),
F.avg("COST_VOICE_ONNET_PMIN2").alias("COST_VOICE_ONNET_PMIN2"),
F.avg("COST_VOICE_ONNET_PMIN3").alias("COST_VOICE_ONNET_PMIN3"),
F.avg("COST_VOICE_ONNET_PMIN4").alias("COST_VOICE_ONNET_PMIN4"),
F.avg("COST_VOICE_OFFNET_PMIN2").alias("COST_VOICE_OFFNET_PMIN2"),
F.avg("COST_VOICE_OFFNET_PMIN3").alias("COST_VOICE_OFFNET_PMIN3"),
F.avg("COST_VOICE_OFFNET_PMIN4").alias("COST_VOICE_OFFNET_PMIN4"),
F.avg("COST_VOICE_VIP_PMIN2").alias("COST_VOICE_VIP_PMIN2"),
F.avg("COST_VOICE_VIP_PMIN3").alias("COST_VOICE_VIP_PMIN3"),
F.avg("COST_VOICE_VIP_PMIN4").alias("COST_VOICE_VIP_PMIN4"),
F.avg("COST_VOICE_TELE2_PMIN2").alias("COST_VOICE_TELE2_PMIN2"),
F.avg("COST_VOICE_TELE2_PMIN3").alias("COST_VOICE_TELE2_PMIN3"),
F.avg("COST_VOICE_TELE2_PMIN4").alias("COST_VOICE_TELE2_PMIN4"),
F.avg("COST_VOICE_TOMATO_PMIN2").alias("COST_VOICE_TOMATO_PMIN2"),
F.avg("COST_VOICE_TOMATO_PMIN3").alias("COST_VOICE_TOMATO_PMIN3"),
F.avg("COST_VOICE_TOMATO_PMIN4").alias("COST_VOICE_TOMATO_PMIN4"),
F.avg("COST_VOICE_BONBON_PMIN2").alias("COST_VOICE_BONBON_PMIN2"),
F.avg("COST_VOICE_BONBON_PMIN3").alias("COST_VOICE_BONBON_PMIN3"),
F.avg("COST_VOICE_BONBON_PMIN4").alias("COST_VOICE_BONBON_PMIN4"),
F.avg("COST_VOICE_FIX_PMIN2").alias("COST_VOICE_FIX_PMIN2"),
F.avg("COST_VOICE_FIX_PMIN3").alias("COST_VOICE_FIX_PMIN3"),
F.avg("COST_VOICE_FIX_PMIN4").alias("COST_VOICE_FIX_PMIN4"),
F.avg("COST_VOICE_INT_PMIN2").alias("COST_VOICE_INT_PMIN2"),
F.avg("COST_VOICE_INT_PMIN3").alias("COST_VOICE_INT_PMIN3"),
F.avg("COST_VOICE_INT_PMIN4").alias("COST_VOICE_INT_PMIN4"),
F.avg("COST_VOICE_ROAMING_PMIN2").alias("COST_VOICE_ROAMING_PMIN2"),
F.avg("COST_VOICE_ROAMING_PMIN3").alias("COST_VOICE_ROAMING_PMIN3"),
F.avg("COST_VOICE_ROAMING_PMIN4").alias("COST_VOICE_ROAMING_PMIN4"),
F.avg("COST_VOICE_TOTAL_PMIN2").alias("COST_VOICE_TOTAL_PMIN2"),
F.avg("COST_VOICE_TOTAL_PMIN3").alias("COST_VOICE_TOTAL_PMIN3"),
F.avg("COST_VOICE_TOTAL_PMIN4").alias("COST_VOICE_TOTAL_PMIN4"),
F.avg("COST_VOICE_VAS_PMIN2").alias("COST_VOICE_VAS_PMIN2"),
F.avg("COST_VOICE_VAS_PMIN3").alias("COST_VOICE_VAS_PMIN3"),
F.avg("COST_VOICE_VAS_PMIN4").alias("COST_VOICE_VAS_PMIN4"),
F.avg("COST_SMS_ONNET_PMIN2").alias("COST_SMS_ONNET_PMIN2"),
F.avg("COST_SMS_ONNET_PMIN3").alias("COST_SMS_ONNET_PMIN3"),
F.avg("COST_SMS_ONNET_PMIN4").alias("COST_SMS_ONNET_PMIN4"),
F.avg("COST_SMS_OFFNET_PMIN2").alias("COST_SMS_OFFNET_PMIN2"),
F.avg("COST_SMS_OFFNET_PMIN3").alias("COST_SMS_OFFNET_PMIN3"),
F.avg("COST_SMS_OFFNET_PMIN4").alias("COST_SMS_OFFNET_PMIN4"),
F.avg("COST_SMS_VIP_PMIN2").alias("COST_SMS_VIP_PMIN2"),
F.avg("COST_SMS_VIP_PMIN3").alias("COST_SMS_VIP_PMIN3"),
F.avg("COST_SMS_VIP_PMIN4").alias("COST_SMS_VIP_PMIN4"),
F.avg("COST_SMS_TELE2_PMIN2").alias("COST_SMS_TELE2_PMIN2"),
F.avg("COST_SMS_TELE2_PMIN3").alias("COST_SMS_TELE2_PMIN3"),
F.avg("COST_SMS_TELE2_PMIN4").alias("COST_SMS_TELE2_PMIN4"),
F.avg("COST_SMS_TOMATO_PMIN2").alias("COST_SMS_TOMATO_PMIN2"),
F.avg("COST_SMS_TOMATO_PMIN3").alias("COST_SMS_TOMATO_PMIN3"),
F.avg("COST_SMS_TOMATO_PMIN4").alias("COST_SMS_TOMATO_PMIN4"),
F.avg("COST_SMS_BONBON_PMIN2").alias("COST_SMS_BONBON_PMIN2"),
F.avg("COST_SMS_BONBON_PMIN3").alias("COST_SMS_BONBON_PMIN3"),
F.avg("COST_SMS_BONBON_PMIN4").alias("COST_SMS_BONBON_PMIN4"),
F.avg("COST_SMS_TOTAL_PMIN2").alias("COST_SMS_TOTAL_PMIN2"),
F.avg("COST_SMS_TOTAL_PMIN3").alias("COST_SMS_TOTAL_PMIN3"),
F.avg("COST_SMS_TOTAL_PMIN4").alias("COST_SMS_TOTAL_PMIN4"),
F.avg("COST_SMS_ROAMING_PMIN2").alias("COST_SMS_ROAMING_PMIN2"),
F.avg("COST_SMS_ROAMING_PMIN3").alias("COST_SMS_ROAMING_PMIN3"),
F.avg("COST_SMS_ROAMING_PMIN4").alias("COST_SMS_ROAMING_PMIN4"),
F.avg("COST_MMS_TOTAL_PMIN2").alias("COST_MMS_TOTAL_PMIN2"),
F.avg("COST_MMS_TOTAL_PMIN3").alias("COST_MMS_TOTAL_PMIN3"),
F.avg("COST_MMS_TOTAL_PMIN4").alias("COST_MMS_TOTAL_PMIN4"),
F.avg("COST_DATA_PMIN2").alias("COST_DATA_PMIN2"),
F.avg("COST_DATA_PMIN3").alias("COST_DATA_PMIN3"),
F.avg("COST_DATA_PMIN4").alias("COST_DATA_PMIN4"),
F.avg("COST_DATA_NATIONAL_PMIN2").alias("COST_DATA_NATIONAL_PMIN2"),
F.avg("COST_DATA_NATIONAL_PMIN3").alias("COST_DATA_NATIONAL_PMIN3"),
F.avg("COST_DATA_NATIONAL_PMIN4").alias("COST_DATA_NATIONAL_PMIN4"))

In [ ]:
mob_09_tele_avg.count()

In [ ]:
L4 = [
'GENDER',
'AGE',
'RATE_PLAN_ID_PMIN2',
'HH_TYPE_PMIN2',
'HH_TYPE_PMIN3',
'HH_TYPE_PMIN4',
'M1_STATUS_PMIN2',
'M1_STATUS_PMIN3',
'M1_STATUS_PMIN4'
]

In [ ]:
mob_09_tele_first = mob_09_tele.select("oib",*L4)

In [ ]:
mob_09_tele_first = mob_09_tele_first.groupBy("oib").agg(
F.first("GENDER").alias("GENDER"),
F.first("AGE").alias("AGE"),
F.first("RATE_PLAN_ID_PMIN2").alias("RATE_PLAN_ID_PMIN2"),
F.first("HH_TYPE_PMIN2").alias("HH_TYPE_PMIN2"),
F.first("HH_TYPE_PMIN3").alias("HH_TYPE_PMIN3"),
F.first("HH_TYPE_PMIN4").alias("HH_TYPE_PMIN4"),
F.first("M1_STATUS_PMIN2").alias("M1_STATUS_PMIN2"),
F.first("M1_STATUS_PMIN3").alias("M1_STATUS_PMIN3"),
F.first("M1_STATUS_PMIN4").alias("M1_STATUS_PMIN4"))

In [ ]:
L5 = [
    'DATA_ONLY_TARIFF',
'HAS_FIXED_VOICE_HH_PMIN2',
'HAS_FIXED_VOICE_HH_PMIN3',
'HAS_FIXED_VOICE_HH_PMIN4',
'HAS_FIXED_BB_HH_PMIN2',
'HAS_FIXED_BB_HH_PMIN3',
'HAS_FIXED_BB_HH_PMIN4',
'HAS_FIXED_TV_HH_PMIN2',
'HAS_FIXED_TV_HH_PMIN3',
'HAS_FIXED_TV_HH_PMIN4',
'FULL_TO_PARTIAL_MAGENTA_PMIN43',
'FULL_TO_PARTIAL_MAGENTA_PMIN32',
'FULL_TO_NO_MAGENTA_PMIN43',
'FULL_TO_NO_MAGENTA_PMIN32',
'PARTIAL_TO_NO_MAGENTA_PMIN43',
'PARTIAL_TO_NO_MAGENTA_PMIN32'
    
]

In [ ]:
mob_09_tele_yes = mob_09_tele.select("oib",*L5)

In [ ]:
mob_09_tele_yes

In [ ]:
mob_09_tele_yes = mob_09_tele_yes.withColumn("DATA_ONLY_TARIFF",F.when((mob_09_tele_yes.DATA_ONLY_TARIFF == "Y") | (mob_09_tele_yes.DATA_ONLY_TARIFF == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_VOICE_HH_PMIN2",F.when((mob_09_tele_yes.HAS_FIXED_VOICE_HH_PMIN2 == "Y") | (mob_09_tele_yes.HAS_FIXED_VOICE_HH_PMIN2 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_VOICE_HH_PMIN3",F.when((mob_09_tele_yes.HAS_FIXED_VOICE_HH_PMIN3 == "Y") | (mob_09_tele_yes.HAS_FIXED_VOICE_HH_PMIN3 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_VOICE_HH_PMIN4",F.when((mob_09_tele_yes.HAS_FIXED_VOICE_HH_PMIN4 == "Y") | (mob_09_tele_yes.HAS_FIXED_VOICE_HH_PMIN4 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_BB_HH_PMIN2",F.when((mob_09_tele_yes.HAS_FIXED_BB_HH_PMIN2 == "Y") | (mob_09_tele_yes.HAS_FIXED_BB_HH_PMIN2 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_BB_HH_PMIN3",F.when((mob_09_tele_yes.HAS_FIXED_BB_HH_PMIN3 == "Y") | (mob_09_tele_yes.HAS_FIXED_BB_HH_PMIN3 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_BB_HH_PMIN4",F.when((mob_09_tele_yes.HAS_FIXED_BB_HH_PMIN4 == "Y") | (mob_09_tele_yes.HAS_FIXED_BB_HH_PMIN4 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_TV_HH_PMIN2",F.when((mob_09_tele_yes.HAS_FIXED_TV_HH_PMIN2 == "Y") | (mob_09_tele_yes.HAS_FIXED_TV_HH_PMIN2 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_TV_HH_PMIN3",F.when((mob_09_tele_yes.HAS_FIXED_TV_HH_PMIN3 == "Y") | (mob_09_tele_yes.HAS_FIXED_TV_HH_PMIN3 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_TV_HH_PMIN4",F.when((mob_09_tele_yes.HAS_FIXED_TV_HH_PMIN4 == "Y") | (mob_09_tele_yes.HAS_FIXED_TV_HH_PMIN4 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_PARTIAL_MAGENTA_PMIN43",F.when((mob_09_tele_yes.FULL_TO_PARTIAL_MAGENTA_PMIN43 == "Y") | (mob_09_tele_yes.FULL_TO_PARTIAL_MAGENTA_PMIN43 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_PARTIAL_MAGENTA_PMIN32",F.when((mob_09_tele_yes.FULL_TO_PARTIAL_MAGENTA_PMIN32 == "Y") | (mob_09_tele_yes.FULL_TO_PARTIAL_MAGENTA_PMIN32 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_NO_MAGENTA_PMIN43",F.when((mob_09_tele_yes.FULL_TO_NO_MAGENTA_PMIN43 == "Y") | (mob_09_tele_yes.FULL_TO_NO_MAGENTA_PMIN43 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_NO_MAGENTA_PMIN32",F.when((mob_09_tele_yes.FULL_TO_NO_MAGENTA_PMIN32 == "Y") | (mob_09_tele_yes.FULL_TO_NO_MAGENTA_PMIN32 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("PARTIAL_TO_NO_MAGENTA_PMIN43",F.when((mob_09_tele_yes.PARTIAL_TO_NO_MAGENTA_PMIN43 == "Y") | (mob_09_tele_yes.PARTIAL_TO_NO_MAGENTA_PMIN43 == "1"),1).otherwise(0))
mob_09_tele_yes = mob_09_tele_yes.withColumn("PARTIAL_TO_NO_MAGENTA_PMIN32",F.when((mob_09_tele_yes.PARTIAL_TO_NO_MAGENTA_PMIN32 == "Y") | (mob_09_tele_yes.PARTIAL_TO_NO_MAGENTA_PMIN32 == "1"),1).otherwise(0))

In [ ]:
# mob_09_tele_yes = mob_09_tele_yes.withColumn("DATA_ONLY_TARIFF",quotes("DATA_ONLY_TARIFF"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_VOICE_HH_PMIN2",quotes("HAS_FIXED_VOICE_HH_PMIN2"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_VOICE_HH_PMIN3",quotes("HAS_FIXED_VOICE_HH_PMIN3"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_VOICE_HH_PMIN4",quotes("HAS_FIXED_VOICE_HH_PMIN4"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_BB_HH_PMIN2",quotes("HAS_FIXED_BB_HH_PMIN2"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_BB_HH_PMIN3",quotes("HAS_FIXED_BB_HH_PMIN3"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_BB_HH_PMIN4",quotes("HAS_FIXED_BB_HH_PMIN4"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_TV_HH_PMIN2",quotes("HAS_FIXED_TV_HH_PMIN2"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_TV_HH_PMIN3",quotes("HAS_FIXED_TV_HH_PMIN3"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("HAS_FIXED_TV_HH_PMIN4",quotes("HAS_FIXED_TV_HH_PMIN4"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_PARTIAL_MAGENTA_PMIN43",quotes("FULL_TO_PARTIAL_MAGENTA_PMIN43"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_PARTIAL_MAGENTA_PMIN32",quotes("FULL_TO_PARTIAL_MAGENTA_PMIN32"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_NO_MAGENTA_PMIN43",quotes("FULL_TO_NO_MAGENTA_PMIN43"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("FULL_TO_NO_MAGENTA_PMIN32",quotes("FULL_TO_NO_MAGENTA_PMIN32"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("PARTIAL_TO_NO_MAGENTA_PMIN43",quotes("PARTIAL_TO_NO_MAGENTA_PMIN43"))
# mob_09_tele_yes = mob_09_tele_yes.withColumn("PARTIAL_TO_NO_MAGENTA_PMIN32",quotes("PARTIAL_TO_NO_MAGENTA_PMIN32"))

In [ ]:
mob_09_tele_yes = mob_09_tele_yes.groupBy("oib").agg(
    
    F.collect_list("DATA_ONLY_TARIFF").alias("DATA_ONLY_TARIFF"),
F.collect_list("HAS_FIXED_VOICE_HH_PMIN2").alias("HAS_FIXED_VOICE_HH_PMIN2"),
F.collect_list("HAS_FIXED_VOICE_HH_PMIN3").alias("HAS_FIXED_VOICE_HH_PMIN3"),
F.collect_list("HAS_FIXED_VOICE_HH_PMIN4").alias("HAS_FIXED_VOICE_HH_PMIN4"),
F.collect_list("HAS_FIXED_BB_HH_PMIN2").alias("HAS_FIXED_BB_HH_PMIN2"),
F.collect_list("HAS_FIXED_BB_HH_PMIN3").alias("HAS_FIXED_BB_HH_PMIN3"),
F.collect_list("HAS_FIXED_BB_HH_PMIN4").alias("HAS_FIXED_BB_HH_PMIN4"),
F.collect_list("HAS_FIXED_TV_HH_PMIN2").alias("HAS_FIXED_TV_HH_PMIN2"),
F.collect_list("HAS_FIXED_TV_HH_PMIN3").alias("HAS_FIXED_TV_HH_PMIN3"),
F.collect_list("HAS_FIXED_TV_HH_PMIN4").alias("HAS_FIXED_TV_HH_PMIN4"),
F.collect_list("FULL_TO_PARTIAL_MAGENTA_PMIN43").alias("FULL_TO_PARTIAL_MAGENTA_PMIN43"),
F.collect_list("FULL_TO_PARTIAL_MAGENTA_PMIN32").alias("FULL_TO_PARTIAL_MAGENTA_PMIN32"),
F.collect_list("FULL_TO_NO_MAGENTA_PMIN43").alias("FULL_TO_NO_MAGENTA_PMIN43"),
F.collect_list("FULL_TO_NO_MAGENTA_PMIN32").alias("FULL_TO_NO_MAGENTA_PMIN32"),
F.collect_list("PARTIAL_TO_NO_MAGENTA_PMIN43").alias("PARTIAL_TO_NO_MAGENTA_PMIN43"),
F.collect_list("PARTIAL_TO_NO_MAGENTA_PMIN32").alias("PARTIAL_TO_NO_MAGENTA_PMIN32")).withColumn("DATA_ONLY_TARIFF",YESorNO("DATA_ONLY_TARIFF")).withColumn("HAS_FIXED_VOICE_HH_PMIN2",YESorNO("HAS_FIXED_VOICE_HH_PMIN2")).withColumn("HAS_FIXED_VOICE_HH_PMIN3",YESorNO("HAS_FIXED_VOICE_HH_PMIN3")).withColumn("HAS_FIXED_VOICE_HH_PMIN4",YESorNO("HAS_FIXED_VOICE_HH_PMIN4")).withColumn("HAS_FIXED_BB_HH_PMIN2",YESorNO("HAS_FIXED_BB_HH_PMIN2")).withColumn("HAS_FIXED_BB_HH_PMIN3",YESorNO("HAS_FIXED_BB_HH_PMIN3")).withColumn("HAS_FIXED_BB_HH_PMIN4",YESorNO("HAS_FIXED_BB_HH_PMIN4")).withColumn("HAS_FIXED_TV_HH_PMIN2",YESorNO("HAS_FIXED_TV_HH_PMIN2")).withColumn("HAS_FIXED_TV_HH_PMIN3",YESorNO("HAS_FIXED_TV_HH_PMIN3")).withColumn("HAS_FIXED_TV_HH_PMIN4",YESorNO("HAS_FIXED_TV_HH_PMIN4")).withColumn("FULL_TO_PARTIAL_MAGENTA_PMIN43",YESorNO("FULL_TO_PARTIAL_MAGENTA_PMIN43")).withColumn("FULL_TO_PARTIAL_MAGENTA_PMIN32",YESorNO("FULL_TO_PARTIAL_MAGENTA_PMIN32")).withColumn("FULL_TO_NO_MAGENTA_PMIN43",YESorNO("FULL_TO_NO_MAGENTA_PMIN43")).withColumn("FULL_TO_NO_MAGENTA_PMIN32",YESorNO("FULL_TO_NO_MAGENTA_PMIN32")).withColumn("PARTIAL_TO_NO_MAGENTA_PMIN43",YESorNO("PARTIAL_TO_NO_MAGENTA_PMIN43")).withColumn("PARTIAL_TO_NO_MAGENTA_PMIN32",YESorNO("PARTIAL_TO_NO_MAGENTA_PMIN32"))

In [ ]:
mob_09_tele_first.count()

In [ ]:
mob_09_tele_max.count()

In [ ]:
mob_09_tele_avg.count()

In [ ]:
mob_09_tele_yes.count()

In [ ]:
mob_09_tele_1 = mob_09_tele_max.join(mob_09_tele_first,["oib"],"left")
mob_09_tele_1 = mob_09_tele_1.join(mob_09_tele_avg,["oib"],"left")
mob_09_tele_1 = mob_09_tele_1.join(mob_09_tele_yes,["oib"],"left")

In [ ]:
mob_09_tele_1.count()

In [ ]:
  
mob_09_tele_1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.jan_31_mob_scoring_202012 as select * from myTemp11")


In [ ]:
BI_OIB = dataImport("sbx_adhoc_ds"," business_id_connection_oib_20210121")

In [ ]:
BI_OIB.printSchema()

In [ ]:
BI_OIB.drop_duplicates(["business_id","oib"]).count()

In [ ]:
BI_OIB = BI_OIB.filter(F.col("oib") !="")

In [ ]:
BI_OIB = BI_OIB.filter(F.col("oib") != "OIB")

In [ ]:
sept_nps = dataImport("cdl_common"," HT_recommended_value_dec")

In [ ]:
# sept_nps = dataImport("cdl_common"," OA_recommended_value_dec")

In [ ]:
sept_nps.count()

In [ ]:
sept =sept_nps.join(BI_OIB,BI_OIB.business_id == sept_nps.oneapp_user_id,"inner")

In [ ]:
sept

In [ ]:
oneapp = dataImport("cdl_common","  Oneapp_dec_31")

In [ ]:
oneapp = oneapp.drop("NPS_value")

In [ ]:
oneapp1 = sept.join(oneapp,["oneapp_user_id"],"left")

In [ ]:
oneapp1.count()

In [ ]:
oneapp1.printSchema()

In [ ]:
oneapp1 = oneapp1.drop("ATTR_value_NUM")
oneapp1 = oneapp1.drop("NPS_value")

In [ ]:
w = Window.partitionBy('oib').orderBy(F.desc('event_timestamp'))
oneapp1 = oneapp1.select('*', row_number().over(w).alias("Rank"))

In [ ]:
oneapp1 = oneapp1.filter(oneapp1.Rank == 1)

In [ ]:
oneapp1 = oneapp1.drop("oneapp_user_id","business_id","event_timestamp","Rank")

In [ ]:

oneapp1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_oib_nps_202012_oa as select * from myTemp11")


In [ ]:
oib_nps_202009 = dataImport("cdl_common"," dec_31_oib_nps_202012_oa")

In [ ]:
fix_scoring_202009 = dataImport("cdl_common"," dec_31_fix_scoring_202012")

In [ ]:
mob_scoring_202009 = dataImport("cdl_common"," dec_31_mob_scoring_202012")

In [ ]:
len(fix_scoring_202009.columns)

In [ ]:
len(mob_scoring_202009.columns)

In [ ]:
oib_nps_202009.count()

In [ ]:
Fix_Mob_NPS_oib_sept  = oib_nps_202009.join(fix_scoring_202009,["oib"],"left")

In [ ]:
Fix_Mob_NPS_oib_sept = Fix_Mob_NPS_oib_sept.join(mob_scoring_202009,["oib"],"left")

In [ ]:
len(Fix_Mob_NPS_oib_sept.columns)

In [ ]:
len(oib_nps_202009.columns)

In [ ]:

Fix_Mob_NPS_oib_sept.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.dec_31_Fix_Mob_NPS_oib_dec_oa as select * from myTemp11")
